In [201]:
import pandas as pd
import numpy as np
import pulp
from pulp import *
import datetime
import matplotlib.pyplot as plt

import FinOpsCodeDeck as finops

from IPython.display import Markdown as md


# 1

In [202]:
'''
Data Import for Current Term Structure
---
Imports all active treasury bonds data, time indexes them by year
'''
data_prompt = pd.read_excel('Table.xlsx', sheet_name='PromptUse', index_col = 'DateDue')
data_prompt = data_prompt/1000
term_structure_df = pd.read_excel('TableNew.xlsx')
term_structure_df['px_last'] = [i if i>=5 else 100 - i for i in term_structure_df['ASKED'].to_list()]
term_structure_df['ttm'] = [(i - datetime.datetime.now())/datetime.timedelta(days=365) for i in term_structure_df['MATURITY']]

In [204]:
'''
Bootstrap yield curve
---
begins with zero-coupon bonds to payout (ttm < 0.5 yrs) & calculates yield
moves to coupon bonds and uses calculated yields to bootstrap further
sorts all bonds into data frame indexed by ttm (by 100th of a year)
NOTE: Averages yields for the same time period
NOTE: assumes yield of period prior if yield for desired period does not exist
'''

'''short term rates'''
mats = []
round_to = 2
for bond_tenor in term_structure_df[term_structure_df['ttm'] <= 0.5].index:
    bond = term_structure_df.loc[bond_tenor]
    cpn = bond['COUPON']/2
    ttm = bond['ttm']
    px = bond['px_last']
    mats.append([np.round(ttm,round_to),np.log((100 + cpn) / bond['px_last']) / bond['ttm']])
rates = pd.DataFrame(mats, columns=['ttm','rate']).set_index('ttm').groupby('ttm').mean()

'''longer term rates'''
for bond_tenor in term_structure_df[term_structure_df['ttm']>=0.5].index:
    bond = term_structure_df.loc[bond_tenor]
    px = bond['px_last']
    ttm = bond['ttm']
    cpn = bond['COUPON']/2
    pmts = int(np.ceil(ttm * 2))
    cfs = [cpn if i+1<pmts else 100 + cpn for i in range(pmts)]
    cfs_idx = [np.round(ttm-i*0.5, round_to) for i in reversed(range(pmts))]
    known_rates = [rates[:cfs_idx[i]].iloc[-1,0] for i in range(pmts-1)]
    val = px - sum([cpn * np.exp((-1) * known_rates[i] * cfs_idx[i]) for i in range(pmts-1)])
    yld = (-1) * (np.log(val / (100+cpn)) / cfs_idx[pmts-1])
    add_df = pd.DataFrame([np.round(ttm, round_to), yld], index=['ttm','rate']).transpose().set_index('ttm')
    rates = pd.concat([rates,add_df],ignore_index=False)
    rates = rates.groupby('ttm').mean()

In [205]:
'''
plot yield curve
---
plots yield curve in percentage points
'''
plt.plot(rates[0.5:] * 100)
plt.title('Bootstrapped Yield Curve (rates per annum)')
plt.xlabel('Tenor')
plt.ylabel('Yield in Percentage Points')

Text(0, 0.5, 'Yield in Percentage Points')

In [206]:
rates

rate
ttm             
-0.00  -0.982162
 0.01   0.859077
 0.02   0.318742
 0.03   0.253033
 0.04   0.198510
...          ...
 28.77  0.031789
 29.02  0.031633
 29.27  0.031208
 29.52  0.031094
 29.77  0.030827

[235 rows x 1 columns]

In [207]:
data_prompt = pd.read_excel('Table.xlsx', sheet_name='PromptUse', index_col = 'DateDue')
data_prompt['ttm'] = np.round((data_prompt.index - datetime.datetime.now()) / datetime.timedelta(days=365), round_to)

r = []
for ttm in data_prompt['ttm']:
    r.append(rates[:ttm].iloc[-1,0])
data_prompt['rates'] = r

npv = sum([data_prompt.iloc[i,0]*np.exp((-1) * data_prompt.iloc[i,1] * data_prompt.iloc[i,2]) for i in range(len(data_prompt))])
dur = sum([data_prompt.iloc[i,1]*data_prompt.iloc[i,0]*np.exp((-1) * data_prompt.iloc[i,1] * data_prompt.iloc[i,2]) for i in range(len(data_prompt))])
con = sum([data_prompt.iloc[i,1]*(data_prompt.iloc[i,1]+1)*data_prompt.iloc[i,0]*np.exp((-1) * (data_prompt.iloc[i,1] + 2 ) * data_prompt.iloc[i,2]) for i in range(len(data_prompt))])

In [208]:
# Using markdown outputs instead of just printing

md('''
<center>

### Solutions

The Net Present Value of the Liabilities is $\${:.2f}$ MM

The Macauley Duration of the Liability stream is ${:.2f}$ years

The Convexity of the Liability stream is ${:.2f}$


'''.format(npv/1000,dur/npv,con/npv))


<center>

### Solutions

The Net Present Value of the Liabilities is $\$117.48$ MM

The Macauley Duration of the Liability stream is $3.95$ years

The Convexity of the Liability stream is $23.29$




# P2

In [244]:
'''
code block
'''
ref_data = ['T ' + '{:.3f}'.format(term_structure_df.iloc[bond,1]) + ' ' + term_structure_df.iloc[bond,0].strftime('%m/%d/%y') for bond in term_structure_df.index]
term_structure_df['ref_data'] = ref_data

bonds_clean = term_structure_df[['ref_data', 'px_last', 'ASKED YIELD']]
bonds_clean.columns = ['Bond', 'Price', 'Yield']
# bonds_clean = bonds_clean.assign(ttm = term_structure_df.ttm.round(2))
bonds_clean = bonds_clean.set_index('Bond')

In [210]:
# You cannot tabe anything within the string literal for the markdown output, See the cell titled "BAD MARKDOWN OUTPUT"
# Also when we finally convert to PDF I plan on using the metadata and hiding the markdown output cell because it takes up to much space and is ugly
# Because of this, have all output cells in their own cell as done here
md('''
<center>

{}

    
'''.format(bonds_clean.to_markdown(colalign = ("center",)))
)


<center>

|       Bond       |   Price |   Yield |
|:----------------:|--------:|--------:|
| T 1.75 05/15/22  | 100.01  | -1.9931 |
| T 2.125 05/15/22 | 100.012 | -2.5447 |
|  T 0.0 05/17/22  |  99.352 |  0.657  |
|  T 0.0 05/19/22  |  99.34  |  0.669  |
|  T 0.0 05/24/22  |  99.32  |  0.69   |
|  T 0.0 05/26/22  |  99.322 |  0.687  |
| T 0.125 05/31/22 |  99.312 |  0.6946 |
| T 1.75 05/31/22  | 100.014 |  0.608  |
| T 1.875 05/31/22 | 100.016 |  0.544  |
|  T 0.0 05/31/22  |  99.307 |  0.702  |
|  T 0.0 06/02/22  |  99.327 |  0.682  |
|  T 0.0 06/07/22  |  99.352 |  0.657  |
|  T 0.0 06/09/22  |  99.347 |  0.662  |
|  T 0.0 06/14/22  |  99.35  |  0.659  |
| T 1.75 06/15/22  | 100.03  |  0.608  |
|  T 0.0 06/16/22  |  99.372 |  0.637  |
|  T 0.0 06/21/22  |  99.342 |  0.667  |
|  T 0.0 06/23/22  |  99.307 |  0.703  |
|  T 0.0 06/28/22  |  99.275 |  0.736  |
| T 0.125 06/30/22 |  99.294 |  0.755  |
| T 1.75 06/30/22  | 100.04  |  0.74   |
| T 2.125 06/30/22 | 100.052 |  0.799  |
|  T 0.0 06/30/22  |  99.275 |  0.736  |
|  T 0.0 07/05/22  |  99.26  |  0.751  |
|  T 0.0 07/07/22  |  99.235 |  0.777  |
|  T 0.0 07/12/22  |  99.237 |  0.774  |
|  T 0.0 07/14/22  |  99.222 |  0.789  |
| T 1.75 07/15/22  | 100.052 |  0.756  |
|  T 0.0 07/21/22  |  99.21  |  0.802  |
|  T 0.0 07/28/22  |  99.205 |  0.807  |
| T 0.125 07/31/22 |  99.27  |  0.871  |
| T 1.875 07/31/22 | 100.064 |  0.902  |
|  T 2.0 07/31/22  | 100.072 |  0.915  |
|  T 0.0 08/04/22  |  99.085 |  0.93   |
|  T 0.0 08/11/22  |  99.055 |  0.96   |
|  T 1.5 08/15/22  | 100.04  |  0.999  |
| T 1.625 08/15/22 | 100.05  |  0.999  |
| T 7.25 08/15/22  | 101.184 |  0.942  |
|  T 0.0 08/18/22  |  99.01  |  1.006  |
|  T 0.0 08/25/22  |  99.01  |  1.007  |
| T 0.125 08/31/22 |  99.234 |  1.042  |
| T 1.625 08/31/22 | 100.052 |  1.057  |
| T 1.875 08/31/22 | 100.076 |  1.037  |
|  T 0.0 09/01/22  |  98.957 |  1.06   |
|  T 0.0 09/08/22  |  98.912 |  1.106  |
|  T 1.5 09/15/22  | 100.03  |  1.214  |
|  T 0.0 09/15/22  |  98.9   |  1.12   |
|  T 0.0 09/22/22  |  98.885 |  1.135  |
|  T 0.0 09/29/22  |  98.887 |  1.133  |
| T 0.125 09/30/22 |  99.184 |  1.258  |
| T 1.75 09/30/22  | 100.056 |  1.266  |
| T 1.875 09/30/22 | 100.074 |  1.244  |
|  T 0.0 10/06/22  |  98.845 |  1.176  |
|  T 0.0 10/13/22  |  98.757 |  1.266  |
| T 1.375 10/15/22 | 100.004 |  1.336  |
|  T 0.0 10/20/22  |  98.732 |  1.292  |
|  T 0.0 10/27/22  |  98.697 |  1.329  |
| T 0.125 10/31/22 |  99.14  |  1.365  |
| T 1.875 10/31/22 | 100.072 |  1.375  |
|  T 2.0 10/31/22  | 100.09  |  1.379  |
|  T 0.0 11/03/22  |  98.675 |  1.352  |
|  T 0.0 11/10/22  |  98.605 |  1.424  |
| T 1.625 11/15/22 | 100.022 |  1.483  |
| T 7.625 11/15/22 | 103.032 |  1.346  |
| T 0.125 11/30/22 |  99.08  |  1.522  |
|  T 2.0 11/30/22  | 100.084 |  1.505  |
|  T 0.0 12/01/22  |  98.64  |  1.389  |
| T 1.625 12/15/22 | 100.012 |  1.557  |
|  T 0.0 12/29/22  |  98.515 |  1.52   |
| T 0.137 12/31/22 |  99.016 |  1.653  |
| T 2.125 12/31/22 | 100.09  |  1.669  |
|  T 1.5 01/15/23  |  99.274 |  1.712  |
|  T 0.0 01/26/23  |  98.452 |  1.586  |
| T 0.125 01/31/23 |  98.27  |  1.771  |
| T 1.75 01/31/23  | 100     |  1.749  |
| T 2.375 01/31/23 | 100.142 |  1.739  |
| T 1.375 02/15/23 |  99.22  |  1.795  |
|  T 2.0 02/15/23  | 100.05  |  1.788  |
| T 7.125 02/15/23 | 104.006 |  1.708  |
|  T 0.0 02/23/23  |  98.332 |  1.713  |
| T 0.125 02/28/23 |  98.206 |  1.855  |
|  T 1.5 02/28/23  |  99.232 |  1.849  |
| T 2.625 02/28/23 | 100.192 |  1.853  |
|  T 0.5 03/15/23  |  98.276 |  1.879  |
|  T 0.0 03/23/23  |  98.282 |  1.772  |
| T 0.125 03/31/23 |  98.134 |  1.954  |
|  T 1.5 03/31/23  |  99.19  |  1.97   |
|  T 2.5 03/31/23  | 100.146 |  1.964  |
| T 0.25 04/15/23  |  98.13  |  2.016  |
|  T 0.0 04/20/23  |  98.127 |  1.933  |
| T 0.125 04/30/23 |  98.056 |  2.057  |
| T 1.625 04/30/23 |  99.19  |  2.056  |
| T 2.75 04/30/23  | 100.206 |  2.061  |
| T 0.125 05/15/23 |  98.03  |  2.066  |
| T 1.75 05/15/23  |  99.22  |  2.068  |
| T 0.125 05/31/23 |  97.306 |  2.115  |
| T 1.625 05/31/23 |  99.156 |  2.12   |
| T 2.75 05/31/23  | 100.204 |  2.124  |
| T 0.25 06/15/23  |  97.316 |  2.136  |
| T 0.125 06/30/23 |  97.23  |  2.19   |
| T 1.375 06/30/23 |  99.034 |  2.181  |
| T 2.625 06/30/23 | 100.156 |  2.178  |
| T 0.125 07/15/23 |  97.19  |  2.227  |
| T 0.125 07/31/23 |  97.144 |  2.271  |
| T 1.25 07/31/23  |  98.262 |  2.243  |
| T 2.75 07/31/23  | 100.19  |  2.248  |
| T 0.125 08/15/23 |  97.114 |  2.277  |
|  T 2.5 08/15/23  | 100.094 |  2.257  |
| T 6.25 08/15/23  | 104.296 |  2.23   |
| T 0.125 08/31/23 |  97.062 |  2.344  |
| T 1.375 08/31/23 |  98.26  |  2.313  |
| T 2.75 08/31/23  | 100.176 |  2.31   |
| T 0.125 09/15/23 |  97.04  |  2.33   |
| T 0.25 09/30/23  |  97.056 |  2.347  |
| T 1.375 09/30/23 |  98.222 |  2.345  |
| T 2.875 09/30/23 | 100.23  |  2.339  |
| T 0.125 10/15/23 |  96.276 |  2.389  |
| T 0.375 10/31/23 |  97.032 |  2.412  |
| T 1.625 10/31/23 |  98.286 |  2.399  |
| T 2.875 10/31/23 | 100.21  |  2.413  |
| T 0.25 11/15/23  |  96.262 |  2.425  |
| T 2.75 11/15/23  | 100.156 |  2.413  |
|  T 0.5 11/30/23  |  97.01  |  2.475  |
| T 2.125 11/30/23 |  99.172 |  2.431  |
| T 2.875 11/30/23 | 100.202 |  2.454  |
| T 0.125 12/15/23 |  96.126 |  2.46   |
| T 0.75 12/31/23  |  97.07  |  2.508  |
| T 2.25 12/31/23  |  99.202 |  2.481  |
| T 2.625 12/31/23 | 100.08  |  2.466  |
| T 0.125 01/15/24 |  96.046 |  2.5    |
| T 0.875 01/31/24 |  97.072 |  2.542  |
|  T 2.5 01/31/24  |  99.302 |  2.532  |
| T 0.125 02/15/24 |  95.286 |  2.534  |
| T 2.75 02/15/24  | 100.112 |  2.542  |
|  T 1.5 02/29/24  |  98.056 |  2.546  |
| T 2.125 02/29/24 |  99.092 |  2.533  |
| T 2.375 02/29/24 |  99.232 |  2.531  |
| T 0.25 03/15/24  |  95.286 |  2.556  |
| T 2.125 03/31/24 |  99.07  |  2.554  |
| T 2.25 03/31/24  |  99.13  |  2.576  |
| T 0.375 04/15/24 |  95.286 |  2.584  |
|  T 2.0 04/30/24  |  98.286 |  2.581  |
| T 2.25 04/30/24  |  99.12  |  2.579  |
|  T 2.5 04/30/24  |  99.27  |  2.582  |
| T 0.25 05/15/24  |  95.144 |  2.601  |
|  T 2.5 05/15/24  |  99.25  |  2.613  |
|  T 2.0 05/31/24  |  98.264 |  2.593  |
| T 0.25 06/15/24  |  95.066 |  2.628  |
| T 1.75 06/30/24  |  98.06  |  2.632  |
|  T 2.0 06/30/24  |  98.214 |  2.647  |
| T 0.375 07/15/24 |  95.064 |  2.669  |
| T 1.75 07/31/24  |  98.014 |  2.665  |
| T 2.125 07/31/24 |  98.264 |  2.674  |
| T 0.375 08/15/24 |  94.31  |  2.693  |
| T 2.375 08/15/24 |  99.092 |  2.702  |
| T 1.25 08/31/24  |  96.25  |  2.708  |
| T 1.875 08/31/24 |  98.042 |  2.721  |
| T 0.375 09/15/24 |  94.224 |  2.735  |
|  T 1.5 09/30/24  |  97.07  |  2.717  |
| T 2.125 09/30/24 |  98.21  |  2.713  |
| T 0.625 10/15/24 |  95.034 |  2.723  |
|  T 1.5 10/31/24  |  97.02  |  2.745  |
| T 2.25 10/31/24  |  98.274 |  2.733  |
| T 0.75 11/15/24  |  95.054 |  2.752  |
| T 2.25 11/15/24  |  98.244 |  2.765  |
|  T 7.5 11/15/24  | 111.176 |  2.685  |
|  T 1.5 11/30/24  |  96.292 |  2.766  |
| T 2.125 11/30/24 |  98.142 |  2.763  |
|  T 1.0 12/15/24  |  95.204 |  2.758  |
| T 1.75 12/31/24  |  97.134 |  2.775  |
| T 2.25 12/31/24  |  98.22  |  2.771  |
| T 1.125 01/15/25 |  95.25  |  2.777  |
| T 1.375 01/31/25 |  96.114 |  2.779  |
|  T 2.5 01/31/25  |  99.09  |  2.776  |
|  T 1.5 02/15/25  |  96.184 |  2.798  |
|  T 2.0 02/15/25  |  97.296 |  2.786  |
| T 7.625 02/15/25 | 112.286 |  2.725  |
| T 1.125 02/28/25 |  95.186 |  2.779  |
| T 2.75 02/28/25  |  99.286 |  2.787  |
| T 1.75 03/15/25  |  97.052 |  2.798  |
|  T 0.5 03/31/25  |  93.222 |  2.798  |
| T 2.625 03/31/25 |  99.176 |  2.787  |
| T 2.625 04/15/25 |  99.164 |  2.799  |
| T 0.375 04/30/25 |  93.04  |  2.815  |
| T 2.875 04/30/25 | 100.066 |  2.8    |
| T 2.125 05/15/25 |  98.016 |  2.806  |
| T 2.75 05/15/25  |  99.282 |  2.791  |
| T 0.25 05/31/25  |  92.186 |  2.811  |
| T 2.875 05/31/25 | 100.066 |  2.802  |
| T 0.25 06/30/25  |  92.116 |  2.819  |
| T 2.75 06/30/25  |  99.266 |  2.805  |
| T 0.25 07/31/25  |  92.046 |  2.826  |
| T 2.875 07/31/25 | 100.06  |  2.813  |
|  T 2.0 08/15/25  |  97.136 |  2.833  |
| T 6.875 08/15/25 | 112.206 |  2.778  |
| T 0.25 08/31/25  |  91.29  |  2.844  |
| T 2.75 08/31/25  |  99.23  |  2.839  |
| T 0.25 09/30/25  |  91.224 |  2.846  |
|  T 3.0 09/30/25  | 100.166 |  2.836  |
| T 0.25 10/31/25  |  91.16  |  2.85   |
|  T 3.0 10/31/25  | 100.156 |  2.849  |
| T 2.25 11/15/25  |  98.004 |  2.85   |
| T 0.375 11/30/25 |  91.224 |  2.855  |
| T 2.875 11/30/25 | 100.026 |  2.849  |
| T 0.375 12/31/25 |  91.166 |  2.853  |
| T 2.625 12/31/25 |  99.09  |  2.835  |
| T 0.375 01/31/26 |  91.094 |  2.864  |
| T 2.625 01/31/26 |  99.062 |  2.855  |
| T 1.625 02/15/26 |  95.194 |  2.867  |
|  T 6.0 02/15/26  | 111.052 |  2.84   |
|  T 0.5 02/28/26  |  91.18  |  2.865  |
|  T 2.5 02/28/26  |  98.232 |  2.856  |
| T 0.75 03/31/26  |  92.092 |  2.867  |
| T 2.25 03/31/26  |  97.24  |  2.867  |
| T 0.75 04/30/26  |  92.042 |  2.868  |
| T 2.375 04/30/26 |  98.062 |  2.861  |
| T 1.625 05/15/26 |  95.094 |  2.879  |
| T 0.75 05/31/26  |  91.292 |  2.884  |
| T 2.125 05/31/26 |  97.05  |  2.875  |
| T 0.875 06/30/26 |  92.074 |  2.886  |
| T 1.875 06/30/26 |  96.052 |  2.868  |
| T 0.625 07/31/26 |  91.024 |  2.891  |
| T 1.875 07/31/26 |  96.024 |  2.87   |
|  T 1.5 08/15/26  |  94.144 |  2.896  |
| T 6.75 08/15/26  | 115.122 |  2.878  |
| T 0.75 08/31/26  |  91.134 |  2.89   |
| T 1.375 08/31/26 |  93.302 |  2.885  |
| T 0.875 09/30/26 |  91.246 |  2.89   |
| T 1.625 09/30/26 |  94.262 |  2.894  |
| T 1.125 10/31/26 |  92.21  |  2.893  |
| T 1.625 10/31/26 |  94.234 |  2.893  |
|  T 2.0 11/15/26  |  96.07  |  2.903  |
|  T 6.5 11/15/26  | 115.034 |  2.893  |
| T 1.25 11/30/26  |  93.032 |  2.882  |
| T 1.625 11/30/26 |  94.222 |  2.879  |
| T 1.25 12/31/26  |  92.304 |  2.889  |
| T 1.75 12/31/26  |  95.034 |  2.887  |
|  T 1.5 01/31/27  |  93.292 |  2.891  |
| T 2.25 02/15/27  |  97.042 |  2.9    |
| T 6.625 02/15/27 | 116.114 |  2.912  |
| T 1.125 02/28/27 |  92.052 |  2.888  |
| T 1.875 02/28/27 |  95.172 |  2.878  |
| T 0.625 03/31/27 |  89.244 |  2.891  |
|  T 2.5 03/31/27  |  98.09  |  2.88   |
|  T 0.5 04/30/27  |  89.002 |  2.897  |
| T 2.75 04/30/27  |  99.126 |  2.881  |
| T 2.375 05/15/27 |  97.164 |  2.913  |
|  T 0.5 05/31/27  |  88.24  |  2.916  |
|  T 0.5 06/30/27  |  88.184 |  2.916  |
| T 0.375 07/31/27 |  87.274 |  2.903  |
| T 2.25 08/15/27  |  96.24  |  2.922  |
| T 6.375 08/15/27 | 116.224 |  2.92   |
|  T 0.5 08/31/27  |  88.06  |  2.926  |
|  T 0.5 10/31/27  |  87.262 |  2.932  |
| T 2.25 11/15/27  |  96.186 |  2.927  |
| T 6.125 11/15/27 | 116.04  |  2.928  |
| T 0.625 11/30/27 |  88.082 |  2.937  |
| T 0.625 12/31/27 |  88.02  |  2.944  |
| T 0.75 01/31/28  |  88.166 |  2.949  |
| T 2.75 02/15/28  |  99.004 |  2.937  |
| T 1.125 02/29/28 |  90.136 |  2.934  |
| T 1.25 03/31/28  |  90.284 |  2.95   |
| T 1.25 04/30/28  |  90.24  |  2.955  |
| T 2.875 05/15/28 |  99.192 |  2.948  |
| T 1.25 05/31/28  |  90.2   |  2.956  |
| T 1.25 06/30/28  |  90.146 |  2.965  |
|  T 1.0 07/31/28  |  88.296 |  2.965  |
| T 2.875 08/15/28 |  99.162 |  2.961  |
|  T 5.5 08/15/28  | 114.156 |  2.944  |
| T 1.125 08/31/28 |  89.164 |  2.964  |
| T 1.25 09/30/28  |  90.026 |  2.968  |
| T 1.375 10/31/28 |  90.226 |  2.966  |
| T 3.125 11/15/28 | 100.312 |  2.959  |
| T 5.25 11/15/28  | 113.192 |  2.935  |
|  T 1.5 11/30/28  |  91.112 |  2.964  |
| T 1.375 12/31/28 |  90.194 |  2.946  |
| T 1.75 01/31/29  |  92.234 |  2.951  |
| T 2.625 02/15/29 |  98.01  |  2.948  |
| T 5.25 02/15/29  | 114.03  |  2.933  |
| T 1.875 02/28/29 |  93.152 |  2.942  |
| T 2.375 03/31/29 |  96.14  |  2.951  |
| T 2.875 04/30/29 |  99.172 |  2.949  |
| T 2.375 05/15/29 |  96.134 |  2.945  |
| T 1.625 08/15/29 |  91.192 |  2.918  |
| T 6.125 08/15/29 | 120.284 |  2.908  |
| T 1.75 11/15/29  |  92.076 |  2.909  |
|  T 1.5 02/15/30  |  90.076 |  2.915  |
| T 0.625 05/15/30 |  83.23  |  2.923  |
| T 6.25 05/15/30  | 123.232 |  2.904  |
| T 0.625 08/15/30 |  83.09  |  2.92   |
| T 0.875 11/15/30 |  84.224 |  2.921  |
| T 1.125 02/15/31 |  86.076 |  2.918  |
| T 5.375 02/15/31 | 118.256 |  2.923  |
| T 1.625 05/15/31 |  89.25  |  2.925  |
| T 1.25 08/15/31  |  86.162 |  2.925  |
| T 1.375 11/15/31 |  87.046 |  2.935  |
| T 1.875 02/15/32 |  91.02  |  2.936  |
| T 2.875 05/15/32 |  99.176 |  2.927  |
|  T 4.5 02/15/36  | 118.09  |  2.88   |
| T 4.75 02/15/37  | 121.096 |  2.957  |
|  T 5.0 05/15/37  | 124.052 |  2.989  |
| T 4.375 02/15/38 | 116.29  |  3.018  |
|  T 4.5 05/15/38  | 118.154 |  3.033  |
|  T 3.5 02/15/39  | 105.026 |  3.108  |
| T 4.25 05/15/39  | 115.006 |  3.106  |
|  T 4.5 08/15/39  | 118.124 |  3.114  |
| T 4.375 11/15/39 | 116.14  |  3.146  |
| T 4.625 02/15/40 | 120.014 |  3.143  |
| T 1.125 05/15/40 |  70.18  |  3.309  |
| T 4.375 05/15/40 | 116.1   |  3.178  |
| T 1.125 08/15/40 |  70.03  |  3.324  |
| T 3.875 08/15/40 | 108.282 |  3.227  |
| T 1.375 11/15/40 |  73.066 |  3.326  |
| T 4.25 11/15/40  | 113.294 |  3.243  |
| T 1.875 02/15/41 |  79.282 |  3.325  |
| T 4.75 02/15/41  | 121.106 |  3.225  |
| T 2.25 05/15/41  |  84.262 |  3.335  |
| T 4.375 05/15/41 | 115.164 |  3.272  |
| T 1.75 08/15/41  |  77.146 |  3.348  |
| T 3.75 08/15/41  | 106.144 |  3.294  |
| T 3.125 11/15/41 |  97.11  |  3.311  |
|  T 2.0 11/30/41  |  80.292 |  3.34   |
| T 2.375 02/15/42 |  86.136 |  3.316  |
| T 3.125 02/15/42 |  97.094 |  3.312  |
|  T 3.0 05/15/42  |  95.104 |  3.322  |
| T 2.75 08/15/42  |  91.074 |  3.349  |
| T 2.75 11/15/42  |  91.022 |  3.356  |
| T 3.125 02/15/43 |  96.232 |  3.345  |
| T 2.875 05/15/43 |  92.264 |  3.354  |
| T 3.625 08/15/43 | 104.124 |  3.335  |
| T 3.75 11/15/43  | 106.112 |  3.334  |
| T 3.625 02/15/44 | 104.09  |  3.346  |
| T 3.375 05/15/44 | 100.094 |  3.356  |
| T 3.125 08/15/44 |  96.11  |  3.359  |
|  T 3.0 11/15/44  |  94.084 |  3.366  |
|  T 2.5 02/15/45  |  86.084 |  3.369  |
|  T 3.0 05/15/45  |  94.094 |  3.358  |
| T 2.875 08/15/45 |  92.116 |  3.35   |
|  T 3.0 11/15/45  |  94.174 |  3.337  |
|  T 2.5 02/15/46  |  86.076 |  3.344  |
|  T 2.5 05/15/46  |  86.074 |  3.338  |
| T 2.25 08/15/46  |  82.016 |  3.335  |
| T 2.875 11/15/46 |  92.234 |  3.31   |
|  T 3.0 02/15/47  |  94.266 |  3.307  |
|  T 3.0 05/15/47  |  94.282 |  3.302  |
| T 2.75 08/15/47  |  90.276 |  3.285  |
| T 2.75 11/15/47  |  90.302 |  3.277  |
|  T 3.0 02/15/48  |  95.21  |  3.25   |
| T 3.125 05/15/48 |  98.04  |  3.232  |
|  T 3.0 08/15/48  |  95.242 |  3.241  |
| T 3.375 11/15/48 | 103.004 |  3.205  |
|  T 3.0 02/15/49  |  96.142 |  3.199  |
| T 2.875 05/15/49 |  94.084 |  3.194  |
| T 2.25 08/15/49  |  82.304 |  3.191  |
| T 2.375 11/15/49 |  85.126 |  3.175  |
|  T 2.0 02/15/50  |  78.066 |  3.189  |
| T 1.25 05/15/50  |  64.106 |  3.184  |
| T 1.375 08/15/50 |  66.176 |  3.177  |
| T 1.625 11/15/50 |  71.07  |  3.166  |
| T 1.875 02/15/51 |  75.31  |  3.152  |
| T 2.375 05/15/51 |  85.156 |  3.141  |
|  T 2.0 08/15/51  |  78.106 |  3.137  |
| T 1.875 11/15/51 |  76.014 |  3.124  |
| T 2.25 02/15/52  |  83.134 |  3.108  |

    


# P3

In [211]:
'''
Data Manipulation
'''
term_by_maturity = term_structure_df.set_index('MATURITY')
possibilities = term_by_maturity.drop(
    index=[i for i in term_by_maturity.index.to_list() if i > data_prompt.index.to_list()[-1]],
    columns=['BID', 'ASKED', 'ASKED YIELD']#, 'ttm']
    )

'''List of bond maturities less than liability maturity'''
date_lists_to_change_to_periods = [
    [i for i in possibilities.index.to_list() if i <= t]
    for t in data_prompt.index.tolist()
    ]
'''Removing the duplicates from each one'''
for i in reversed(range(1,len(date_lists_to_change_to_periods))):
    for j in range(0,len(date_lists_to_change_to_periods[i-1])):
        date_lists_to_change_to_periods[i].remove(date_lists_to_change_to_periods[i-1][j])

for i in range(0,len(date_lists_to_change_to_periods)):
    possibilities.loc[date_lists_to_change_to_periods[i],'period'] = i+1

possibilities['face'] = 100
possibilities['bond#'] = range(1,len(possibilities)+1)
possibilities = possibilities.set_index('bond#')

'''for labeling later'''
dec_var_names = possibilities['ref_data']

In [212]:
'''Getting data ready for the solver'''

'''Exmpty Array'''
cfs = np.zeros((len(possibilities),len(date_lists_to_change_to_periods)))

'''CF Matrix'''
'''Will make function later'''
for i in range(0, len(cfs)):
    for j in range(1, len(cfs[0])+1):
        if possibilities.loc[i+1,'period'] == j and possibilities.loc[i+1,'COUPON'] == 0:
            cfs[i][j-1] = possibilities.loc[i+1,'face']
        elif possibilities.loc[i+1,'period'] == j and possibilities.loc[i+1,'COUPON'] != 0:
            cfs[i][0:j-1] = possibilities.loc[i+1,'COUPON']/2
            cfs[i][j-1] = possibilities.loc[i+1,'face'] + possibilities.loc[i+1,'COUPON']/2

cf_matrix = cfs.tolist()
prices = possibilities['px_last'].values.tolist()
liabilities = data_prompt['Amount'].values.tolist()

In [213]:
'''Solving for the dedicated portfolio'''

# Making variable list of strings
# bonds = ['$b_{}$'.format('{'+str(i)+'}') for i in range(1, len(cf_matrix)+1)]
periods = [i for i in range(0,len(cf_matrix[0])+1)]

# Must reverse both price vector and cashflow matrix
cf_matrix
prices

# Dictionary of period constraints
period_dict = {}
for i in range(0,len(cf_matrix[0])):
    period_dict['Period {}'.format(i+1)] = dict(zip(dec_var_names,[cf_matrix[j][i] for j in range(0,len(cf_matrix))]))

objective = dict(zip(dec_var_names, prices))

# Decision Vars
quantity = LpVariable.dict('', dec_var_names, lowBound=0)
excess = LpVariable.dict('carryover', periods, lowBound=0)

# Intializing the Problem
dedication_1 = LpProblem('Dedicated', LpMinimize)

# Objective function
dedication_1 += excess[0]+lpSum([objective[i]*quantity[i] for i in dec_var_names])

# Constraints
for i in range(0,len(cf_matrix[0])):
    dedication_1 += lpSum([period_dict['Period {}'.format(i+1)][j]*quantity[j] for j in dec_var_names]) + excess[i] - excess[i+1] == liabilities[i]

dedication_1.solve()

1

In [214]:
composition = pd.DataFrame(
    [v.varValue for v in dedication_1.variables() if v.name[0] != 'c' and v.varValue > 0],
    index=[v.name[1:] for v in dedication_1.variables() if v.name[0] != 'c' and v.varValue > 0],
    columns=['Quantity']
    )
# Just have to match bonds with the tickers Jack created instead

'''Printing Solutions'''

md(''' 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \${:.2f} $ MM <br>

</center>

<center>

{}

'''.format(dedication_1.objective.value()/1000,composition.to_markdown(colalign = ("right",))))

 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \$117.77 $ MM <br>

</center>

<center>

|                  |   Quantity |
|-----------------:|-----------:|
| T_0.625_05_15_30 |    69.7819 |
|  T_0.75_05_31_26 |   107.674  |
|  T_1.25_05_31_28 |    63.7436 |
|   T_1.5_11_30_24 |    71.4316 |
|   T_2.0_05_31_24 |    50.9223 |
| T_2.875_04_30_29 |    66.4559 |
|   T_5.5_08_15_28 |    84.142  |
| T_6.125_08_15_29 |    77.4112 |
|  T_6.25_08_15_23 |    59.0762 |
| T_6.375_08_15_27 |    42.3924 |
| T_6.625_02_15_27 |    50.7125 |
|  T_6.75_08_15_26 |    78.0774 |
| T_6.875_08_15_25 |    75.0923 |
| T_7.125_02_15_23 |    66.7    |
|  T_7.25_08_15_22 |    83.6671 |
| T_7.625_02_15_25 |    81.9673 |



# P4

In [215]:
'''
Pull sensitivity analysis
---
https://s3.amazonaws.com/assets.datacamp.com/production/course_8835/slides/chapter4.pdf
'''
o = [{'name':name, 'shadow price':c.pi} for name, c in dedication_1.constraints.items()]
shadow_px = pd.DataFrame(o).set_index(data_prompt.index).drop('name',axis=1)
clean_shadow_px = pd.DataFrame(o).set_index(data_prompt.index.strftime('%m/%d/%y')).drop('name',axis=1)
md(''' 
##### <center> Sensitivity Analysis - Shadow Prices  </center>

<center>

{}

'''.format(clean_shadow_px.to_markdown(colalign = ("right",)))
)

 
##### <center> Sensitivity Analysis - Shadow Prices  </center>

<center>

|   DateDue |   shadow price |
|----------:|---------------:|
|  12/15/22 |       0.976444 |
|  06/15/23 |       0.970693 |
|  12/15/23 |       0.952351 |
|  06/15/24 |       0.944203 |
|  12/15/24 |       0.927139 |
|  06/15/25 |       0.906415 |
|  12/15/25 |       0.896101 |
|  06/15/26 |       0.884951 |
|  12/15/26 |       0.870135 |
|  06/15/27 |       0.856877 |
|  12/15/27 |       0.8426   |
|  06/15/28 |       0.834112 |
|  12/15/28 |       0.820296 |
|  06/15/29 |       0.812113 |
|  12/15/29 |       0.795826 |
|  06/15/30 |       0.788304 |



In [216]:
'''
Presents implied and actual yield curve as a plot
'''
shadow_ttm = ((shadow_px.index - datetime.datetime.now()) / datetime.timedelta(days=365)).to_list()
shadow_factors = shadow_px['shadow price'].to_list()
implied_rates = [-np.log(shadow_factors[i]) / shadow_ttm[i] for i in range(len(shadow_ttm))]

implied_rates_df = pd.DataFrame(
                        data = [shadow_ttm, implied_rates],
                        index=['ttm','implied_rate']
                    )

implied_rates_df = (implied_rates_df
                        .transpose()
                        .round({'ttm':round_to})
                        .set_index('ttm')
                    )

plt.plot(rates[0.5:8.5] * 100)
plt.plot(implied_rates_df * 100)
plt.title('Bootstrapped and Implied Yield Curve (rates per annum)')
plt.xlabel('Tenor')
plt.ylabel('Yield in Percentage Points')
plt.legend(['Actual', 'Implied'])

In [217]:
'''

'''

md(''' 
##### <center> Sensitivity Analysis - Shadow Prices  </center>

<center>

{}

'''.format((implied_rates_df*100).to_markdown(colalign = ("right",)))
)

 
##### <center> Sensitivity Analysis - Shadow Prices  </center>

<center>

|   ttm |   implied_rate |
|------:|---------------:|
|  0.58 |        4.08849 |
|  1.08 |        2.74987 |
|  1.58 |        3.08402 |
|  2.08 |        2.75443 |
|  2.59 |        2.92568 |
|  3.08 |        3.18562 |
|  3.59 |        3.05935 |
|  4.08 |        2.99241 |
|  4.59 |        3.03342 |
|  5.08 |        3.03793 |
|  5.59 |        3.06604 |
|  6.09 |        2.97983 |
|  6.59 |        3.00658 |
|  7.09 |        2.93653 |
|  7.59 |        3.00947 |
|  8.09 |        2.94134 |



# P5

In [218]:
'''
Aggregates cashflow matrix and ref data for immunization
---
Puts cashflow matrix into a dataframe for merging
merges possible bond ref data with cashflow matrix
cleans resulting dataframe

NOTE: MATH NEEDS WORK HERE BUT WE CAN FIGURE OUT
from here: use ttm and col_num against calculated curve to find appropriate measure
        pv_factor = exp{-rt} = exp{- () * (ttm)}
'''
cf_df = pd.DataFrame(cf_matrix, index=dec_var_names.to_list())

cf_df = pd.merge(                                                           # Combines possible bonds with cashflow matrix
    left = possibilities,                                                   # possible bonds - SAME DF AS DEDICATION
    right = cf_df,                                                          # Cashflow matrix - NP ARRAY FROM DEDICATION AS DF FOR MERGING
    how='inner',                                                            # Catches any missed bonds on merge
    left_on='ref_data',                                                     # possibilities not indexed by bond name - CHAZ IS THIS SOMETHING WE CAN ADJUST OR NO?????????????????????
    right_index=True                                                        # Casflow df indexed by bond name
)

cf_df = (cf_df
            .drop(['COUPON','period','face'],axis=1)                        # Drops unnecessary ref data
            .set_index('ref_data')                                          # Sets index to bond name
            .round({'ttm':round_to})                                        # rounds time to maturity to 2 decimal places -- allows use of derived term structure (indexed by hundredths)
        )

cf_df

px_last   ttm         0       1       2       3       4  \
ref_data                                                                    
T 1.75 05/15/22   100.010 -0.00  100.8750  0.0000  0.0000  0.0000  0.0000   
T 2.125 05/15/22  100.012 -0.00  101.0625  0.0000  0.0000  0.0000  0.0000   
T 0.0 05/17/22     99.352  0.00  100.0000  0.0000  0.0000  0.0000  0.0000   
T 0.0 05/19/22     99.340  0.01  100.0000  0.0000  0.0000  0.0000  0.0000   
T 0.0 05/24/22     99.320  0.02  100.0000  0.0000  0.0000  0.0000  0.0000   
...                   ...   ...       ...     ...     ...     ...     ...   
T 6.125 08/15/29  120.284  7.25    3.0625  3.0625  3.0625  3.0625  3.0625   
T 1.75 11/15/29    92.076  7.51    0.8750  0.8750  0.8750  0.8750  0.8750   
T 1.5 02/15/30     90.076  7.76    0.7500  0.7500  0.7500  0.7500  0.7500   
T 0.625 05/15/30   83.230  8.00    0.3125  0.3125  0.3125  0.3125  0.3125   
T 6.25 05/15/30   123.232  8.00    3.1250  3.1250  3.1250  3.1250  3.1250   

                       5       6       7       8       9      10      11  \
ref_data                                                                   
T 1.75 05/15/22   0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
T 2.125 05/15/22  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
T 0.0 05/17/22    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
T 0.0 05/19/22    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
T 0.0 05/24/22    0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
...                  ...     ...     ...     ...     ...     ...     ...   
T 6.125 08/15/29  3.0625  3.0625  3.0625  3.0625  3.0625  3.0625  3.0625   
T 1.75 11/15/29   0.8750  0.8750  0.8750  0.8750  0.8750  0.8750  0.8750   
T 1.5 02/15/30    0.7500  0.7500  0.7500  0.7500  0.7500  0.7500  0.7500   
T 0.625 05/15/30  0.3125  0.3125  0.3125  0.3125  0.3125  0.3125  0.3125   
T 6.25 05/15/30   3.1250  3.1250  3.1250  3.1250  3.1250  3.1250  3.1250   

                      12      13        14        15  
ref_data                                              
T 1.75 05/15/22   0.0000  0.0000    0.0000    0.0000  
T 2.125 05/15/22  0.0000  0.0000    0.0000    0.0000  
T 0.0 05/17/22    0.0000  0.0000    0.0000    0.0000  
T 0.0 05/19/22    0.0000  0.0000    0.0000    0.0000  
T 0.0 05/24/22    0.0000  0.0000    0.0000    0.0000  
...                  ...     ...       ...       ...  
T 6.125 08/15/29  3.0625  3.0625  103.0625    0.0000  
T 1.75 11/15/29   0.8750  0.8750  100.8750    0.0000  
T 1.5 02/15/30    0.7500  0.7500    0.7500  100.7500  
T 0.625 05/15/30  0.3125  0.3125    0.3125  100.3125  
T 6.25 05/15/30   3.1250  3.1250    3.1250  103.1250  

[289 rows x 18 columns]

In [219]:
'''
Create PResent Value, Duration, and Convexity factors for all possible time index based on derived rates curve
'''
t = rates.index
r = rates['rate']
npv_factor = np.exp(-r*t)
dur_factor = (t / (1+r)) * np.exp(-r*t)
con_factor = ((t*(t+1))/ (1+r)**2) * np.exp(-r*t)

In [220]:
'''
Calculates npv, duration, and convexity terms for all bonds consiuder in problem
'''
npvs=[]
durs=[]
cons=[]
for bond in cf_df.index:
    bond_df = cf_df.loc[bond]
    bond_ttm = bond_df.loc['ttm']
    bond_cf_stream = bond_df.loc[0:]
    eo_cfs = bond_cf_stream.idxmax()
    cpn_ttm = [(bond_ttm - 0.5*i).round(round_to) for i in range(eo_cfs+1)]
    bond_cf_ttm = pd.Series(data=bond_df.loc[0:eo_cfs].to_list(), index=reversed(cpn_ttm))

    bond_npv = sum([bond_cf_ttm.loc[i] * npv_factor.loc[:i].iloc[-1] for i in bond_cf_ttm.index])
    bond_dur = sum([bond_cf_ttm.loc[i] * dur_factor.loc[:i].iloc[-1] for i in bond_cf_ttm.index])
    bond_con = sum([bond_cf_ttm.loc[i] * con_factor.loc[:i].iloc[-1] for i in bond_cf_ttm.index])

    npvs.append(bond_npv)
    durs.append(bond_dur)
    cons.append(bond_con)

immunization_df = pd.DataFrame([npvs, durs, cons], columns=cf_df.index, index=['npv','duration','convexity']).transpose()

In [221]:
'''
Solves immunization portfolio
'''
bond_count = LpVariable.dicts('Bonds',dec_var_names,lowBound=0)

immunization = LpProblem('immunization', LpMinimize)

immunization += lpSum([cf_df['px_last'].loc[i] * bond_count[i] for i in dec_var_names])
immunization += lpSum([immunization_df['npv'].loc[i] * bond_count[i] for i in dec_var_names]) == npv
immunization += lpSum([immunization_df['duration'].loc[i] * bond_count[i] for i in dec_var_names]) == dur
immunization += lpSum([immunization_df['convexity'].loc[i] * bond_count[i] for i in dec_var_names]) == con

immunization.solve()

1

In [222]:
'''
Print Solution to Immunized portfolio
'''
bonds_fin = pd.DataFrame([[v.name[6:] for v in immunization.variables()], [v.varValue for v in immunization.variables()]], index=['bond','amt']).transpose().set_index('bond')
bonds_fin[bonds_fin['amt']>0]

md(
''' 
##### <center> Portoflio Allocation - Immunization  </center>

<center> Immunized Portfolio Value of ${:.2f} MM  </center> <br>


<center>

{}

'''.format(value(immunization.objective.value()) / 1000, bonds_fin[bonds_fin['amt']>0].to_markdown(colalign = ("right",))))

 
##### <center> Portoflio Allocation - Immunization  </center>

<center> Immunized Portfolio Value of $116.84 MM  </center> <br>


<center>

|             bond |     amt |
|-----------------:|--------:|
| T_6.125_08_15_29 | 169.86  |
| T_6.625_02_15_27 | 726.856 |
|  T_7.25_08_15_22 | 118.686 |



# Problem 6

In [223]:
'''
Immunization part
Calculates npv, duration, and convexity terms for all bonds considered in the problem
FROM period 7-16
'''

ded_period = 6
imm_period = 5
imm_start_period = len(data_prompt) - imm_period*2
imm_end_period = len(data_prompt)

In [224]:
'''
Solves combined portfolio
'''
bond_q = LpVariable.dicts('Bond',dec_var_names,lowBound=0)
excess_cf = LpVariable.dicts('ExcessCf', periods[:ded_period+1], lowBound=0)


combined = LpProblem('Combined', LpMinimize)

combined += lpSum([cf_df['px_last'][i] * bond_q[i] for i in dec_var_names]) + excess_cf[0] 

for i in range(0,ded_period):
    combined += lpSum([cf_df[i][j]*bond_q[j] for j in dec_var_names]) + excess_cf[i]- excess_cf[i+1] == liabilities[i]

combined += lpSum([immunization_df['npv'][i] * bond_q[i] for i in dec_var_names]) == npv
combined += lpSum([immunization_df['duration'][i] * bond_q[i] for i in dec_var_names]) == dur
combined += lpSum([immunization_df['convexity'][i] * bond_q[i] for i in dec_var_names]) == con

combined.solve()

1

In [225]:
'''
Print Solution to Combined portfolio
'''
bonds_comb = pd.DataFrame( 
            [v.varValue for v in combined.variables() if v.varValue > 0], 
            index=[str(v.name[:-8].replace('Bond', '').replace('_', ' ') + v.name[-8:].replace('_', '/'))
            if v.name[0] == 'B' else str(r'$\text{Excess}_' + v.name[-1] + '$')
            for v in combined.variables() if v.varValue > 0], 
            columns=['Quantity'])

bonds_comb.index.name = 'Bonds/Excess Cashflows'



In [226]:
md(''' 
##### <center> Portoflio Allocation - Combined  </center>

<center> Combined Portfolio Value of ${:.2f} MM  </center> <br>


<center>

{}

'''.format(value(combined.objective) / 1000, bonds_comb.to_markdown(colalign = ("right",)))
)

 
##### <center> Portoflio Allocation - Combined  </center>

<center> Combined Portfolio Value of $119.42 MM  </center> <br>


<center>

|   Bonds/Excess Cashflows |   Quantity |
|-------------------------:|-----------:|
|         T 0.125 12/15/23 |    76.0327 |
|          T 0.25 05/31/25 |    96.5936 |
|          T 0.25 06/15/23 |    85.9253 |
|          T 0.25 06/15/24 |    66.0802 |
|          T 0.75 01/31/28 |   658.426  |
|           T 1.0 12/15/24 |    86.1628 |
|          T 1.25 04/30/28 |   130.644  |
|         T 1.625 12/15/22 |    87.6766 |
|        $\text{Excess}_0$ |  1753.63   |



# P7

I thought the best way to solve this was to think of the trade balance as the objective. If we are going to buy and simultaniously short a bucket of bonds, and hold to each their maturities, then means $\text{excess}_i$ is not a decision variable anymore and $\text{excess}_0$ is a given by the new decision variables. A new decision variable arises and that is the "excess spent". 

We can think of what was excess as now our reserve funds from the trade balance and it gets diminished every time the decision variable "excess spent" is used. The constraints are pretty straight forward from there.

Why this works?
- Ans: I believe these new constraint bound the objective. Are we essentialy using the objective inside of the constraints, yes, but I do not think that violates any rules

In [227]:
'''Getting data how I want it'''
data = term_structure_df
Bid_p = data['BID'].to_list()[0:len(cf_matrix)]
Ask_p = data['ASKED'].to_list()[0:len(cf_matrix)]
Bid_p_fix = [Bid_p[i] 
            if Bid_p[i] > 10 else 100 - Bid_p[i]
            for i in range(0,len(Bid_p))]
Ask_p_fix = [Ask_p[i] 
            if Ask_p[i] > 10 else 100 - Ask_p[i]
            for i in range(0,len(Ask_p))]

In [228]:
'''Solving for the dedicated portfolio'''

# Making variable list of strings
periods = [i for i in range(0,len(cf_matrix[0])+1)]
excess_from_arb_left = [0*i for i in periods]


# Dictionary of period constraints
period_dict = {}
for i in range(0,len(cf_matrix[0])):
    period_dict['Period {}'.format(i+1)] = dict(zip(dec_var_names,[cf_matrix[j][i] for j in range(0,len(cf_matrix))]))

# Dictionary with tickers and prices for both bid and ask
bids = dict(zip(dec_var_names, Bid_p_fix))
asks = dict(zip(dec_var_names, Ask_p_fix))

# Decision Vars

quantity_s = LpVariable.dict('S', dec_var_names, lowBound=0)                # Qauntity Short
quantity_l = LpVariable.dict('L', dec_var_names, lowBound=0)                # Qauntity Long
excess_spent = LpVariable.dict('reserve_used', periods, lowBound=0)     # Reserves spent at time t


# Intializing the Problem
dedication_2 = LpProblem('Shorts', LpMinimize)

# Objective function
dedication_2 += lpSum([asks[i]*quantity_l[i] for i in dec_var_names]) - lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) + excess_spent[0]

# Constraints

for i in range(0,len(cf_matrix[0])):
    # Liabilties constraint
    dedication_2 += lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_l[j] for j in dec_var_names]) - lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_s[j] for j in dec_var_names]) + excess_spent[i] - excess_spent[i+1] == liabilities[i]/1000
    # Constraint on the on the reserves at time t                                                                                        #sum of the "excess spent" so far

dedication_2 += lpSum([asks[i]*quantity_l[i] for i in dec_var_names]) - lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) + excess_spent[0]  >= 0
  





In [229]:
dedication_2.solve()
LpStatus[dedication_2.status]



'Optimal'

In [230]:
comp_dates = pd.DataFrame(
    [v.varValue for v in dedication_2.variables() if  v.varValue > 0],
    index=[v.name for v in dedication_2.variables() if v.varValue > 0 ],
    columns=['Quantity']
    )

In [231]:
comp_dates = pd.DataFrame(
    [v.varValue for v in dedication_2.variables() if v.name[2] == 'T' and v.varValue > 0],
    index=[str(v.name[:-8].replace('_', ' ')+v.name[-8:].replace('_', '/')) for v in dedication_2.variables() if v.name[2] == 'T' and v.varValue > 0 ],
    columns=['Quantity']
    )
comp_reserve = pd.DataFrame(
    [v.varValue for v in dedication_2.variables() if v.name[0] == 'r' and v.varValue > 0],
    index=[str(v.name.replace('_', ' ')) for v in dedication_2.variables() if v.name[0] == 'r' and v.varValue > 0 ],
    columns=['Quantity']
    )
'''Manipulating the dataframe so that it will be sorted by maturity'''
maturities_ded_2 = [datetime.datetime.strptime(i[-8:], '%m/%d/%y') for i in comp_dates.index.to_list()]
comp_dates.reset_index(inplace=True)
comp_dates['Dates'] = maturities_ded_2
comp_dates.set_index('Dates',inplace=True)
comp_dates.sort_index(axis=0, inplace=True)
comp_dates.set_index('index', inplace = True)

composition = pd.concat([comp_dates,comp_reserve])

In [232]:
'''Printing Solutions'''
'''Note that the portfolio cost is a negative because our trade balance was positive'''
md(''' 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \${:.2f} $ <br>

</center>

<center>

{}

'''.format(dedication_2.objective.value()*-1000000,composition.to_markdown(colalign = ("right",))))

 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \$79.20 $ <br>

</center>

<center>

|                    |   Quantity |
|-------------------:|-----------:|
|   S T 0.0 06/16/22 |  2.04863   |
| S T 0.137 12/31/22 | 63.2136    |
| L T 7.125 02/15/23 | 61.1449    |
|  L T 6.25 08/15/23 |  0.0563529 |
|   L T 2.0 05/31/24 |  0.0481139 |
|   L T 1.5 11/30/24 |  0.0685951 |
| L T 7.625 02/15/25 |  0.154094  |
|  L T 0.75 05/31/26 |  0.104984  |
|  L T 6.75 08/15/26 |  0.0753781 |
| L T 6.625 02/15/27 |  0.0479221 |
| L T 6.375 08/15/27 |  0.0395095 |
| L T 2.875 05/15/28 |  0.0607689 |
|   L T 5.5 08/15/28 |  0.0816424 |
|  L T 5.25 02/15/29 |  0.0638876 |
| L T 6.125 08/15/29 |  0.0755646 |
|  L T 6.25 05/15/30 |  0.0678788 |
|     reserve used 6 |  7.49844   |



In [233]:
data[data['MATURITY'] == '05-15-30']

MATURITY  COUPON      BID    ASKED  ASKED YIELD  px_last       ttm  \
287 2030-05-15   0.625   83.220   83.230        2.923   83.230  8.002229   
288 2030-05-15   6.250  123.222  123.232        2.904  123.232  8.002229   

             ref_data  
287  T 0.625 05/15/30  
288   T 6.25 05/15/30

In [234]:
data.iloc[0:289]

MATURITY  COUPON      BID    ASKED  ASKED YIELD  px_last       ttm  \
0   2022-05-15   1.750  100.004  100.010      -1.9931  100.010 -0.003251   
1   2022-05-15   2.125  100.006  100.012      -2.5447  100.012 -0.003251   
2   2022-05-17   0.000    0.658    0.648       0.6570   99.352  0.002229   
3   2022-05-19   0.000    0.670    0.660       0.6690   99.340  0.007708   
4   2022-05-24   0.000    0.690    0.680       0.6900   99.320  0.021407   
..         ...     ...      ...      ...          ...      ...       ...   
284 2029-08-15   6.125  120.274  120.284       2.9080  120.284  7.254283   
285 2029-11-15   1.750   92.066   92.076       2.9090   92.076  7.506338   
286 2030-02-15   1.500   90.066   90.076       2.9150   90.076  7.758393   
287 2030-05-15   0.625   83.220   83.230       2.9230   83.230  8.002229   
288 2030-05-15   6.250  123.222  123.232       2.9040  123.232  8.002229   

             ref_data  
0     T 1.75 05/15/22  
1    T 2.125 05/15/22  
2      T 0.0 05/17/22  
3      T 0.0 05/19/22  
4      T 0.0 05/24/22  
..                ...  
284  T 6.125 08/15/29  
285   T 1.75 11/15/29  
286    T 1.5 02/15/30  
287  T 0.625 05/15/30  
288   T 6.25 05/15/30  

[289 rows x 8 columns]

In [235]:
cf_df = pd.DataFrame(
    cf_matrix,
    index = data.iloc[0:289]['ref_data']
)
checker = pd.DataFrame(
    index = ['excess used','inflow', 'outflow', 'Total', 'liabilities'],
    columns=cf_df.columns
)
cf_df.loc['T 0.0 06/16/22',:] = cf_df.loc['T 0.0 06/16/22',:]*-1

cf_df.loc['T 0.137 12/31/22',:] = cf_df.loc['T 0.137 12/31/22',:]*-1

In [236]:
checker

0    1    2    3    4    5    6    7    8    9    10   11   12  \
excess used  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
inflow       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
outflow      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
Total        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
liabilities  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

              13   14   15  
excess used  NaN  NaN  NaN  
inflow       NaN  NaN  NaN  
outflow      NaN  NaN  NaN  
Total        NaN  NaN  NaN  
liabilities  NaN  NaN  NaN

In [237]:
check_cf_df = cf_df.drop([i for i in cf_df.index if i not in [j[2:] for j in composition.iloc[:-1].index.tolist()]])

In [238]:
check_cf_df

0         1         2         3         4         5   \
ref_data                                                                       
T 0.0 06/16/22   -100.0000   -0.0000   -0.0000   -0.0000   -0.0000   -0.0000   
T 0.137 12/31/22   -0.0685 -100.0685   -0.0000   -0.0000   -0.0000   -0.0000   
T 7.125 02/15/23    3.5625  103.5625    0.0000    0.0000    0.0000    0.0000   
T 6.25 08/15/23     3.1250    3.1250  103.1250    0.0000    0.0000    0.0000   
T 2.0 05/31/24      1.0000    1.0000    1.0000  101.0000    0.0000    0.0000   
T 1.5 11/30/24      0.7500    0.7500    0.7500    0.7500  100.7500    0.0000   
T 7.625 02/15/25    3.8125    3.8125    3.8125    3.8125    3.8125  103.8125   
T 0.75 05/31/26     0.3750    0.3750    0.3750    0.3750    0.3750    0.3750   
T 6.75 08/15/26     3.3750    3.3750    3.3750    3.3750    3.3750    3.3750   
T 6.625 02/15/27    3.3125    3.3125    3.3125    3.3125    3.3125    3.3125   
T 6.375 08/15/27    3.1875    3.1875    3.1875    3.1875    3.1875    3.1875   
T 2.875 05/15/28    1.4375    1.4375    1.4375    1.4375    1.4375    1.4375   
T 5.5 08/15/28      2.7500    2.7500    2.7500    2.7500    2.7500    2.7500   
T 5.25 02/15/29     2.6250    2.6250    2.6250    2.6250    2.6250    2.6250   
T 6.125 08/15/29    3.0625    3.0625    3.0625    3.0625    3.0625    3.0625   
T 6.25 05/15/30     3.1250    3.1250    3.1250    3.1250    3.1250    3.1250   

                      6         7         8         9         10        11  \
ref_data                                                                     
T 0.0 06/16/22   -0.0000   -0.0000   -0.0000   -0.0000   -0.0000   -0.0000   
T 0.137 12/31/22 -0.0000   -0.0000   -0.0000   -0.0000   -0.0000   -0.0000   
T 7.125 02/15/23  0.0000    0.0000    0.0000    0.0000    0.0000    0.0000   
T 6.25 08/15/23   0.0000    0.0000    0.0000    0.0000    0.0000    0.0000   
T 2.0 05/31/24    0.0000    0.0000    0.0000    0.0000    0.0000    0.0000   
T 1.5 11/30/24    0.0000    0.0000    0.0000    0.0000    0.0000    0.0000   
T 7.625 02/15/25  0.0000    0.0000    0.0000    0.0000    0.0000    0.0000   
T 0.75 05/31/26   0.3750  100.3750    0.0000    0.0000    0.0000    0.0000   
T 6.75 08/15/26   3.3750    3.3750  103.3750    0.0000    0.0000    0.0000   
T 6.625 02/15/27  3.3125    3.3125    3.3125  103.3125    0.0000    0.0000   
T 6.375 08/15/27  3.1875    3.1875    3.1875    3.1875  103.1875    0.0000   
T 2.875 05/15/28  1.4375    1.4375    1.4375    1.4375    1.4375  101.4375   
T 5.5 08/15/28    2.7500    2.7500    2.7500    2.7500    2.7500    2.7500   
T 5.25 02/15/29   2.6250    2.6250    2.6250    2.6250    2.6250    2.6250   
T 6.125 08/15/29  3.0625    3.0625    3.0625    3.0625    3.0625    3.0625   
T 6.25 05/15/30   3.1250    3.1250    3.1250    3.1250    3.1250    3.1250   

                        12        13        14       15  
ref_data                                                 
T 0.0 06/16/22     -0.0000   -0.0000   -0.0000   -0.000  
T 0.137 12/31/22   -0.0000   -0.0000   -0.0000   -0.000  
T 7.125 02/15/23    0.0000    0.0000    0.0000    0.000  
T 6.25 08/15/23     0.0000    0.0000    0.0000    0.000  
T 2.0 05/31/24      0.0000    0.0000    0.0000    0.000  
T 1.5 11/30/24      0.0000    0.0000    0.0000    0.000  
T 7.625 02/15/25    0.0000    0.0000    0.0000    0.000  
T 0.75 05/31/26     0.0000    0.0000    0.0000    0.000  
T 6.75 08/15/26     0.0000    0.0000    0.0000    0.000  
T 6.625 02/15/27    0.0000    0.0000    0.0000    0.000  
T 6.375 08/15/27    0.0000    0.0000    0.0000    0.000  
T 2.875 05/15/28    0.0000    0.0000    0.0000    0.000  
T 5.5 08/15/28    102.7500    0.0000    0.0000    0.000  
T 5.25 02/15/29     2.6250  102.6250    0.0000    0.000  
T 6.125 08/15/29    3.0625    3.0625  103.0625    0.000  
T 6.25 05/15/30     3.1250    3.1250    3.1250  103.125

In [239]:
for i in periods[:-1]:
    checker.loc['liabilities', i] = liabilities[i]
    checker.loc['Total', i] = np.sum(np.array(check_cf_df[i]) * np.array(comp_dates['Quantity'].to_list()))

In [240]:
checker

0         1     2     3     4          5         6   \
excess used        NaN       NaN   NaN   NaN   NaN        NaN       NaN   
inflow             NaN       NaN   NaN   NaN   NaN        NaN       NaN   
outflow            NaN       NaN   NaN   NaN   NaN        NaN       NaN   
Total        10.999995  8.999925   8.0   7.0   9.0  17.498437  1.501563   
liabilities      11000      9000  8000  7000  9000      10000      9000   

                7     8     9     10    11    12    13    14    15  
excess used    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
inflow         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
outflow        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
Total         12.0   9.0   6.0   5.0   7.0   9.0   7.0   8.0   7.0  
liabilities  12000  9000  6000  5000  7000  9000  7000  8000  7000

In [241]:
cf_df[cf_df.index == [j[2:] for j in composition.iloc[:-1].index.tolist()]]

ValueError: ('Shapes must match', (289,), (16,))

In [ ]:
cf_df.index

Index(['T 1.75 05/15/22', 'T 2.125 05/15/22', 'T 0.0 05/17/22',
       'T 0.0 05/19/22', 'T 0.0 05/24/22', 'T 0.0 05/26/22',
       'T 0.125 05/31/22', 'T 1.75 05/31/22', 'T 1.875 05/31/22',
       'T 0.0 05/31/22',
       ...
       'T 1.875 02/28/29', 'T 2.375 03/31/29', 'T 2.875 04/30/29',
       'T 2.375 05/15/29', 'T 1.625 08/15/29', 'T 6.125 08/15/29',
       'T 1.75 11/15/29', 'T 1.5 02/15/30', 'T 0.625 05/15/30',
       'T 6.25 05/15/30'],
      dtype='object', name='ref_data', length=289)

In [ ]:
for i in periods[:-1]:
    for j in composition.iloc[:-1].index.tolist():
        checker.loc['Total',i] = sum(cf_df.loc[j[2:],i]*composition.loc[j,'Quantity'])
    
        



TypeError: 'numpy.float64' object is not iterable

Found Arbitrage 
not feasible: 63 and 61 mill of one bond


In [ ]:
'''Data Manipulation'''
data = term_structure_df
Bid_p = data['BID'].to_list()[0:len(cf_matrix)]
Ask_p = data['ASKED'].to_list()[0:len(cf_matrix)]
Bid_p_fix = [Bid_p[i] 
            if Bid_p[i] > 10 else 100 - Bid_p[i]
            for i in range(0,len(Bid_p))]
Ask_p_fix = [Ask_p[i] 
            if Ask_p[i] > 10 else 100 - Ask_p[i]
            for i in range(0,len(Ask_p))]
            
'''Solving for the dedicated portfolio'''

# Making variable list of strings
periods = [i for i in range(0,len(cf_matrix[0])+1)]
excess_from_arb_left = [0*i for i in periods]


# Dictionary of period constraints
period_dict = {}
for i in range(0,len(cf_matrix[0])):
    period_dict['Period {}'.format(i+1)] = dict(zip(dec_var_names,[cf_matrix[j][i] for j in range(0,len(cf_matrix))]))

# Dictionary with tickers and prices for both bid and ask
bids = dict(zip(dec_var_names, Bid_p_fix))
asks = dict(zip(dec_var_names, Ask_p_fix))

# Decision Vars

quantity_s = LpVariable.dict('S', dec_var_names, lowBound=0)                # Qauntity Short
quantity_l = LpVariable.dict('L', dec_var_names, lowBound=0)                # Qauntity Long
excess_spent = LpVariable.dict('reserve_used', periods, lowBound=0)     # Reserves spent at time t


# Intializing the Problem
dedication_2 = LpProblem('Shorts', LpMinimize)

# Objective function
dedication_2 +=   lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) - lpSum([asks[i]*quantity_l[i] for i in dec_var_names])

# Constraints

for i in range(0,len(cf_matrix[0])):
    # Liabilties constraint
    dedication_2 += lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_l[j] for j in dec_var_names]) - lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_s[j] for j in dec_var_names]) - excess_spent[i] + excess_spent[i+1] == liabilities[i]/1000
    # Constraint on the on the reserves at time t                                                                                        #sum of the "excess spent" so far
    dedication_2 += lpSum([bids[j]*quantity_s[j] for j in dec_var_names]) - lpSum([asks[j]*quantity_l[j] for j in dec_var_names]) - lpSum([excess_spent[j+1] for j in range(0,i)]) >= 0
    # Constraint stating that the "excess spent" at time t must be <= the current reserves.
    dedication_2 += excess_spent[i+1] <= lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) - lpSum([asks[i]*quantity_l[i] for i in dec_var_names]) - lpSum([excess_spent[j+1] for j in range(0,i)])    

dedication_2.solve()

1

In [ ]:
composition = pd.DataFrame(
    [v.varValue for v in dedication_2.variables() if v.varValue > 0],
    index=[v.name for v in dedication_2.variables() if v.varValue > 0],
    columns=['Quantity']
    )



In [ ]:
dedication_2.objective.value()*-1000000

54.68460593505142

In [ ]:
composition

Quantity
L_T_0.75_05_31_26    0.105428
L_T_1.25_05_31_28    0.061260
L_T_1.5_11_30_24     0.069356
L_T_2.0_05_31_24     0.048867
L_T_5.25_02_15_29    0.063888
L_T_5.5_08_15_28     0.081642
L_T_6.125_08_15_29   0.075565
L_T_6.25_05_15_30    0.067879
L_T_6.25_08_15_23    0.057083
L_T_6.375_08_15_27   0.039985
L_T_6.625_02_15_27   0.048382
L_T_6.75_08_15_26    0.075823
L_T_6.875_08_15_25   0.072921
L_T_7.125_02_15_23  61.101012
L_T_7.625_02_15_25   0.079876
S_T_0.0_06_16_22     2.046371
S_T_0.137_12_31_22  63.167382

In [ ]:
'''Manipulating the dataframe so that it will be sorted by maturity and look nice'''
composition['Dates'] = [datetime.datetime.strptime(i[-8:], '%m/%d/%y') for i in composition.index.to_list()]
composition.reset_index(inplace=True)
composition.set_index('Dates',inplace=True)
composition.sort_index(axis=0, inplace=True)
composition.set_index('index', inplace = True)

'''Printing Solutions'''
'''Note that the portfolio cost is a negative because our trade balance was positive'''
md(''' 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \${:.2f} $ <br>

</center>

<center>

{}

'''.format(dedication_2.objective.value()*-1000000,composition.to_markdown(colalign = ("right",))))

ValueError: time data '05_31_26' does not match format '%m/%d/%y'

In [ ]:
data[data['ref_data'] == 'T 0.0 06/16/22']

MATURITY  COUPON    BID  ASKED  ASKED YIELD  px_last       ttm  \
15 2022-06-16     0.0  0.638  0.628        0.637   99.372  0.085343   

          ref_data  
15  T 0.0 06/16/22

# P7 constraint on the transaction amount

I looked through some data on some auctions and it seems as though we can say no more than 5 mill per transaction because that is really how much 

In [ ]:
'''Solving for the dedicated portfolio'''

# Making variable list of strings
periods = [i for i in range(0,len(cf_matrix[0])+1)]
excess_from_arb_left = [0*i for i in periods]


# Dictionary of period constraints
period_dict = {}
for i in range(0,len(cf_matrix[0])):
    period_dict['Period {}'.format(i+1)] = dict(zip(dec_var_names,[cf_matrix[j][i] for j in range(0,len(cf_matrix))]))

# Dictionary with tickers and prices for both bid and ask
bids = dict(zip(dec_var_names, Bid_p_fix))
asks = dict(zip(dec_var_names, Ask_p_fix))

# Decision Vars

quantity_s = LpVariable.dict('S', dec_var_names, lowBound=0)                # Qauntity Short
quantity_l = LpVariable.dict('L', dec_var_names, lowBound=0)                # Qauntity Long
excess_spent = LpVariable.dict('reserve_used', periods[1:], lowBound=0)     # Reserves spent at time t


# Intializing the Problem
dedication_2_trans_lim = LpProblem('Shorts', LpMaximize)

# Objective function
dedication_2_trans_lim += lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) - lpSum([asks[i]*quantity_l[i] for i in dec_var_names]) - lpSum([excess_spent[i] for i in periods[1:]])

# Constraints

for i in range(0,len(cf_matrix[0])):
    # Liabilties constraint
    dedication_2_trans_lim += lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_l[j] for j in dec_var_names]) - lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_s[j] for j in dec_var_names]) + excess_spent[i+1] == liabilities[i]/1000
    # Constraint on the on the reserves at time t                                                                                        #sum of the "excess spent" so far
    dedication_2_trans_lim += lpSum([bids[j]*quantity_s[j] for j in dec_var_names]) - lpSum([asks[j]*quantity_l[j] for j in dec_var_names]) - lpSum([excess_spent[i] for i in periods[1:]]) >= 0
    # Constraint stating that the "excess spent" at time t must be <= the current reserves.
    dedication_2_trans_lim += excess_spent[i+1] <= lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) - lpSum([asks[i]*quantity_l[i] for i in dec_var_names]) - lpSum([excess_spent[j+1] for j in range(0,i)])   





In [ ]:
dedication_2_trans_lim.solve()
LpStatus[dedication_2_trans_lim.status]

'Unbounded'

In [ ]:
dedication_2_trans_lim.objective.value()

1009.0066099193476

In [ ]:
min(data['COUPON'])

0.0

In [ ]:
composition = pd.DataFrame(
    [v.varValue for v in dedication_2_trans_lim.variables() if v.varValue > 0],
    index=[v.name for v in dedication_2_trans_lim.variables() if v.varValue > 0],
    columns=['Quantity']
    )

In [ ]:
'''Printing Solutions'''
'''Note that the portfolio cost is a negative because our trade balance was positive'''
md(''' 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \${:.2f} $ <br>

</center>

<center>

{}

'''.format(dedication_2_trans_lim.objective.value()*-1000000,composition.to_markdown(colalign = ("right",))))

 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \$-1009006609.92 $ <br>

</center>

<center>

|                    |   Quantity |
|-------------------:|-----------:|
|   L_T_0.0_08_04_22 |  15        |
|   L_T_0.0_08_11_22 |  15        |
|   L_T_0.0_08_18_22 |  15        |
|   L_T_0.0_08_25_22 |  15        |
|   L_T_0.0_09_01_22 |  15        |
|   L_T_0.0_09_08_22 |  15        |
|   L_T_0.0_09_15_22 |  15        |
|   L_T_0.0_09_22_22 |  15        |
|   L_T_0.0_09_29_22 |  15        |
|   L_T_0.0_10_06_22 |  15        |
|   L_T_0.0_10_13_22 |  15        |
|   L_T_0.0_10_20_22 |  15        |
|   L_T_0.0_10_27_22 |  15        |
|   L_T_0.0_11_03_22 |  15        |
|   L_T_0.0_11_10_22 |  15        |
|   L_T_0.0_12_01_22 |  15        |
| L_T_0.125_05_31_23 |  15        |
| L_T_0.125_09_30_22 |  15        |
| L_T_0.125_10_15_23 |  15        |
| L_T_0.125_10_31_22 |  15        |
| L_T_0.125_11_30_22 |  15        |
| L_T_0.125_12_15_23 |  15        |
|  L_T_0.25_05_15_24 |  15        |
|  L_T_0.25_05_31_25 |  15        |
|  L_T_0.25_06_15_23 |  15        |
|  L_T_0.25_06_15_24 |  15        |
|  L_T_0.25_10_31_25 |  15        |
|  L_T_0.25_11_15_23 |  15        |
| L_T_0.375_04_15_24 |  15        |
| L_T_0.375_11_30_25 |  15        |
|   L_T_0.5_03_15_23 |  15        |
|   L_T_0.5_03_31_25 |  15        |
|   L_T_0.5_05_31_27 |  15        |
|   L_T_0.5_10_31_27 |  15        |
| L_T_0.625_03_31_27 |  15        |
| L_T_0.625_05_15_30 |  15        |
|  L_T_0.75_05_31_26 |  15        |
|  L_T_0.75_11_15_24 |  15        |
| L_T_0.875_09_30_26 |  15        |
|   L_T_1.0_12_15_24 |  15        |
| L_T_1.125_10_31_26 |  15        |
|  L_T_1.25_03_31_28 |  15        |
|  L_T_1.25_04_30_28 |  15        |
|  L_T_1.25_05_31_28 |  15        |
|  L_T_1.25_08_31_24 |  15        |
| L_T_1.375_08_31_26 |  15        |
| L_T_1.375_09_30_23 |  15        |
| L_T_1.375_10_31_28 |  15        |
|   L_T_1.5_01_15_23 |  15        |
|   L_T_1.5_01_31_27 |  15        |
|   L_T_1.5_02_28_23 |  15        |
|   L_T_1.5_03_31_23 |  15        |
|   L_T_1.5_11_30_24 |  15        |
| L_T_1.625_02_15_26 |  15        |
| L_T_1.625_04_30_23 |  15        |
| L_T_1.625_05_15_26 |  15        |
| L_T_1.625_05_31_23 |  15        |
| L_T_1.625_09_30_26 |  15        |
| L_T_1.625_10_31_23 |  15        |
| L_T_1.625_10_31_26 |  15        |
| L_T_1.625_11_30_26 |  15        |
|  L_T_1.75_05_15_22 |  15        |
|  L_T_1.75_05_15_23 |  15        |
|  L_T_1.75_05_31_22 |  15        |
|  L_T_1.75_06_15_22 |  15        |
| L_T_1.875_05_31_22 |  15        |
| L_T_1.875_07_31_22 |  15        |
| L_T_1.875_08_31_22 |  15        |
| L_T_1.875_09_30_22 |  15        |
| L_T_1.875_10_31_22 |  15        |
|   L_T_2.0_02_15_25 |  15        |
|   L_T_2.0_04_30_24 |  15        |
|   L_T_2.0_05_31_24 |  15        |
|   L_T_2.0_07_31_22 |  15        |
|   L_T_2.0_10_31_22 |  15        |
|   L_T_2.0_11_15_26 |  13.1191   |
|   L_T_2.0_11_30_22 |  15        |
| L_T_2.125_05_15_22 |  15        |
| L_T_2.125_05_31_26 |  15        |
| L_T_2.125_06_30_22 |  15        |
| L_T_2.125_07_31_24 |  15        |
| L_T_2.125_09_30_24 |  15        |
| L_T_2.125_11_30_23 |  15        |
| L_T_2.125_11_30_24 |  15        |
|  L_T_2.25_03_31_24 |  10.2953   |
|  L_T_2.25_03_31_26 |  15        |
|  L_T_2.25_04_30_24 |  15        |
|  L_T_2.25_08_15_27 |  15        |
|  L_T_2.25_10_31_24 |  15        |
|  L_T_2.25_11_15_24 |  15        |
|  L_T_2.25_11_15_27 |  15        |
|  L_T_2.25_12_31_24 |  15        |
| L_T_2.375_01_31_23 |   7.21611  |
| L_T_2.375_02_29_24 |  15        |
| L_T_2.375_03_31_29 |  14.4181   |
| L_T_2.375_05_15_27 |  15        |
| L_T_2.375_05_15_29 |  15        |
|   L_T_2.5_01_31_24 |  15        |
|   L_T_2.5_02_28_26 |  15        |
|   L_T_2.5_03_31_23 |  15        |
|   L_T_2.5_04_30_24 |  15        |
|   L_T_2.5_05_15_24 |  15        |
| L_T_2.625_02_28_23 |  15        |
| L_T_2.625_03_31_25 |  15        |
| L_T_2.625_04_15_25 |  15        |
|  L_T_2.75_02_28_25 |  15        |
|  L_T_2.75_04_30_23 |  15        |
|  L_T_2.75_04_30_27 |  13.2015   |
|  L_T_2.75_05_15_25 |  15        |
|  L_T_2.75_05_31_23 |  15        |
|  L_T_2.75_06_30_25 |  15        |
|  L_T_2.75_07_31_23 |  15        |
|  L_T_2.75_08_31_23 |  15        |
|  L_T_2.75_08_31_25 |  15        |
|  L_T_2.75_11_15_23 |  15        |
| L_T_2.875_04_30_29 |  15        |
| L_T_2.875_05_15_28 |  13.7444   |
| L_T_2.875_08_15_28 |  15        |
| L_T_2.875_09_30_23 |  15        |
| L_T_2.875_10_31_23 |  15        |
| L_T_2.875_11_30_23 |  15        |
|   L_T_3.0_09_30_25 |  15        |
|   L_T_3.0_10_31_25 |  15        |
| L_T_3.125_11_15_28 |  15        |
|  L_T_5.25_02_15_29 |  15        |
|   L_T_5.5_08_15_28 |  15        |
|   L_T_6.0_02_15_26 |  15        |
| L_T_6.125_08_15_29 |  15        |
|  L_T_6.25_05_15_30 |   0.131515 |
|  L_T_6.25_08_15_23 |  15        |
| L_T_6.375_08_15_27 |  15        |
| L_T_6.625_02_15_27 |  15        |
|  L_T_6.75_08_15_26 |  15        |
| L_T_6.875_08_15_25 |  15        |
| L_T_7.125_02_15_23 |  15        |
|  L_T_7.25_08_15_22 |  15        |
|   L_T_7.5_11_15_24 |  15        |
| L_T_7.625_02_15_25 |  15        |
|   S_T_0.0_01_26_23 |  15        |
|   S_T_0.0_02_23_23 |  15        |
|   S_T_0.0_03_23_23 |  15        |
|   S_T_0.0_04_20_23 |  15        |
|   S_T_0.0_05_17_22 |  15        |
|   S_T_0.0_05_19_22 |  15        |
|   S_T_0.0_05_24_22 |  15        |
|   S_T_0.0_05_26_22 |  15        |
|   S_T_0.0_05_31_22 |  15        |
|   S_T_0.0_06_02_22 |  15        |
|   S_T_0.0_06_07_22 |  15        |
|   S_T_0.0_06_09_22 |  15        |
|   S_T_0.0_06_14_22 |  15        |
|   S_T_0.0_06_16_22 |  15        |
|   S_T_0.0_06_21_22 |  15        |
|   S_T_0.0_06_23_22 |  15        |
|   S_T_0.0_06_28_22 |  15        |
|   S_T_0.0_06_30_22 |  15        |
|   S_T_0.0_07_05_22 |  15        |
|   S_T_0.0_07_07_22 |  15        |
|   S_T_0.0_07_12_22 |  15        |
|   S_T_0.0_07_14_22 |  15        |
|   S_T_0.0_07_21_22 |  15        |
|   S_T_0.0_07_28_22 |  15        |
|   S_T_0.0_12_29_22 |  15        |
| S_T_0.125_01_15_24 |  15        |
| S_T_0.125_01_31_23 |  15        |
| S_T_0.125_02_15_24 |  15        |
| S_T_0.125_02_28_23 |  15        |
| S_T_0.125_03_31_23 |  15        |
| S_T_0.125_04_30_23 |  15        |
| S_T_0.125_05_15_23 |  15        |
| S_T_0.125_05_31_22 |  15        |
| S_T_0.125_06_30_22 |  15        |
| S_T_0.125_06_30_23 |  15        |
| S_T_0.125_07_15_23 |  15        |
| S_T_0.125_07_31_22 |  15        |
| S_T_0.125_07_31_23 |  15        |
| S_T_0.125_08_15_23 |  15        |
| S_T_0.125_08_31_22 |   8.51794  |
| S_T_0.125_08_31_23 |  15        |
| S_T_0.125_09_15_23 |  15        |
| S_T_0.137_12_31_22 |  15        |
|  S_T_0.25_03_15_24 |  15        |
|  S_T_0.25_04_15_23 |  15        |
|  S_T_0.25_06_30_25 |  15        |
|  S_T_0.25_07_31_25 |  15        |
|  S_T_0.25_08_31_25 |  15        |
|  S_T_0.25_09_30_23 |  15        |
|  S_T_0.25_09_30_25 |   3.13183  |
| S_T_0.375_01_31_26 |  15        |
| S_T_0.375_04_30_25 |  15        |
| S_T_0.375_07_15_24 |  15        |
| S_T_0.375_07_31_27 |  15        |
| S_T_0.375_08_15_24 |  15        |
| S_T_0.375_09_15_24 |   4.45637  |
| S_T_0.375_10_31_23 |  15        |
| S_T_0.375_12_31_25 |  15        |
|   S_T_0.5_02_28_26 |  15        |
|   S_T_0.5_06_30_27 |  15        |
|   S_T_0.5_08_31_27 |  15        |
|   S_T_0.5_11_30_23 |  15        |
| S_T_0.625_07_31_26 |  15        |
| S_T_0.625_10_15_24 |  15        |
| S_T_0.625_11_30_27 |   1.56132  |
| S_T_0.625_12_31_27 |  15        |
|  S_T_0.75_01_31_28 |  15        |
|  S_T_0.75_03_31_26 |  15        |
|  S_T_0.75_04_30_26 |  15        |
|  S_T_0.75_08_31_26 |  15        |
|  S_T_0.75_12_31_23 |  15        |
| S_T_0.875_01_31_24 |  15        |
| S_T_0.875_06_30_26 |  15        |
|   S_T_1.0_07_31_28 |  15        |
| S_T_1.125_01_15_25 |  15        |
| S_T_1.125_02_28_25 |  15        |
| S_T_1.125_02_28_27 |  15        |
| S_T_1.125_02_29_28 |  15        |
| S_T_1.125_08_31_28 |  15        |
|  S_T_1.25_06_30_28 |  15        |
|  S_T_1.25_07_31_23 |  15        |
|  S_T_1.25_09_30_28 |  15        |
|  S_T_1.25_11_30_26 |  15        |
|  S_T_1.25_12_31_26 |  15        |
| S_T_1.375_01_31_25 |  15        |
| S_T_1.375_02_15_23 |  15        |
| S_T_1.375_06_30_23 |  15        |
| S_T_1.375_08_31_23 |   6.08722  |
| S_T_1.375_10_15_22 |  15        |
| S_T_1.375_12_31_28 |  15        |
|   S_T_1.5_02_15_25 |  15        |
|   S_T_1.5_02_15_30 |  15        |
|   S_T_1.5_02_29_24 |  15        |
|   S_T_1.5_08_15_22 |  15        |
|   S_T_1.5_08_15_26 |  15        |
|   S_T_1.5_09_15_22 |  15        |
|   S_T_1.5_09_30_24 |  15        |
|   S_T_1.5_10_31_24 |  15        |
| S_T_1.625_08_15_22 |  15        |
| S_T_1.625_08_15_29 |   0.185106 |
| S_T_1.625_08_31_22 |  15        |
| S_T_1.625_11_15_22 |  15        |
| S_T_1.625_12_15_22 |  15        |
|  S_T_1.75_01_31_29 |  15        |
|  S_T_1.75_03_15_25 |  15        |
|  S_T_1.75_06_30_24 |  15        |
|  S_T_1.75_07_15_22 |  15        |
|  S_T_1.75_07_31_24 |  15        |
|  S_T_1.75_09_30_22 |  15        |
|  S_T_1.75_11_15_29 |  15        |
|  S_T_1.75_12_31_24 |  15        |
|  S_T_1.75_12_31_26 |  15        |
| S_T_1.875_02_28_27 |  15        |
| S_T_1.875_02_28_29 |  15        |
| S_T_1.875_06_30_26 |  15        |
| S_T_1.875_07_31_26 |  15        |
| S_T_1.875_08_31_24 |  15        |
|   S_T_2.0_02_15_23 |  15        |
|   S_T_2.0_06_30_24 |  15        |
|   S_T_2.0_08_15_25 |  15        |
| S_T_2.125_02_29_24 |  15        |
| S_T_2.125_03_31_24 |  15        |
| S_T_2.125_05_15_25 |  15        |
| S_T_2.125_12_31_22 |  15        |
|  S_T_2.25_02_15_27 |  15        |
|  S_T_2.25_11_15_25 |  15        |
|  S_T_2.25_12_31_23 |  15        |
| S_T_2.375_04_30_26 |   2.487    |
| S_T_2.375_08_15_24 |  15        |
|   S_T_2.5_01_31_25 |  15        |
|   S_T_2.5_03_31_27 |  15        |
|   S_T_2.5_08_15_23 |  15        |
| S_T_2.625_01_31_26 |  15        |
| S_T_2.625_02_15_29 |  15        |
| S_T_2.625_06_30_23 |  15        |
| S_T_2.625_12_31_23 |  15        |
| S_T_2.625_12_31_25 |  15        |
|  S_T_2.75_02_15_24 |  15        |
|  S_T_2.75_02_15_28 |  15        |
| S_T_2.875_04_30_25 |   3.79848  |
| S_T_2.875_07_31_25 |  15        |
| S_T_2.875_11_30_25 |  15        |
|  S_T_5.25_11_15_28 |   1.15049  |
| S_T_6.125_11_15_27 |  15        |
|   S_T_6.5_11_15_26 |  15        |
| S_T_7.625_11_15_22 |  15        |



# P8

In [ ]:
'''Looking at the implied term structure of the last problem'''
o = [{'name':name, 'shadow price':c.pi} for name, c in dedication_2.constraints.items() if c.pi > 0]


In [ ]:
shadows = pd.DataFrame(
    o,
    index = data_prompt.index.tolist()
).drop(labels=['name'], axis = 1)

In [ ]:
shadows

shadow price
2022-12-15  6.296923e-15
2023-06-15  6.312124e-15
2023-12-15  6.201100e-15
2024-06-15  6.308148e-15
2024-12-15  6.026772e-15
2025-06-15  5.749364e-15
2025-12-15  5.495436e-15
2026-06-15  5.794718e-15
2026-12-15  6.186104e-15
2027-06-15  6.052294e-15
2027-12-15  5.921907e-15
2028-06-15  5.743882e-15
2028-12-15  5.393901e-15
2029-06-15  5.400471e-15
2029-12-15  5.515432e-15
2030-06-15  5.512089e-15

In [ ]:
shadow_ttm = ((shadows.index - datetime.datetime.now()) / datetime.timedelta(days=365)).to_list()
shadow_ttm

[0.5839730849849696,
 1.082603221971271,
 1.5839730849849696,
 2.0853429479986683,
 2.586712811012367,
 3.0853429479986683,
 3.586712811012367,
 4.085342947998668,
 4.586712811012367,
 5.085342947998669,
 5.586712811012367,
 6.088082674026066,
 6.589452537039764,
 7.088082674026066,
 7.589452537039764,
 8.088082674026065]

In [ ]:
looper = shadows.index.tolist()
implied_rates = pd.DataFrame(
    [-np.log(shadows.loc[looper[i],'shadow price'])/shadow_ttm[i] for i in range(len(looper))],
    index = shadow_ttm,
    columns= ['implied rates']
)

In [ ]:
implied_rates

implied rates
0.583973      55.993531
1.082603      30.201558
1.583973      20.653160
2.085343      15.679404
2.586713      12.657982
3.085343      10.627566
3.586713       9.154583
4.085343       8.024254
4.586713       7.132880
5.085343       6.437784
5.586713       5.863934
6.088083       5.386037
6.589453       4.985772
7.088083       4.634863
7.589453       4.325902
8.088083       4.059285

In [ ]:
plt.plot(rates[0.5:8.5] * 100)
plt.plot(implied_rates)
plt.title('Bootstrapped and Implied Yield Curve (rates per annum)')
plt.xlabel('Tenor')
plt.ylabel('Yield in Percentage Points')
plt.legend(['Actual', 'Implied'])

In [ ]:
possibilities

COUPON  px_last       ttm          ref_data  period  face
bond#                                                           
1       1.750  100.010 -0.002328   T 1.75 05/15/22     1.0   100
2       2.125  100.012 -0.002328  T 2.125 05/15/22     1.0   100
3       0.000   99.352  0.003151    T 0.0 05/17/22     1.0   100
4       0.000   99.340  0.008631    T 0.0 05/19/22     1.0   100
5       0.000   99.320  0.022329    T 0.0 05/24/22     1.0   100
...       ...      ...       ...               ...     ...   ...
285     6.125  120.284  7.255206  T 6.125 08/15/29    15.0   100
286     1.750   92.076  7.507261   T 1.75 11/15/29    15.0   100
287     1.500   90.076  7.759316    T 1.5 02/15/30    16.0   100
288     0.625   83.230  8.003151  T 0.625 05/15/30    16.0   100
289     6.250  123.232  8.003151   T 6.25 05/15/30    16.0   100

[289 rows x 6 columns]

In [ ]:
periods_and_ticks = pd.DataFrame(
    possibilities['period'].values.tolist(),
    index = possibilities['ref_data'].values.tolist(),
    columns=['period']
)

In [ ]:
'''Solving for the dedicated portfolio'''

# Making variable list of strings
periods = [i for i in range(0,len(cf_matrix[0])+1)]
excess_from_arb_left = [0*i for i in periods]


# Dictionary of period constraints
period_dict = {}
for i in range(0,len(cf_matrix[0])):
    period_dict['Period {}'.format(i+1)] = dict(zip(dec_var_names,[cf_matrix[j][i] for j in range(0,len(cf_matrix))]))

# Dictionary with tickers and prices for both bid and ask
bids = dict(zip(dec_var_names, Bid_p_fix))
asks = dict(zip(dec_var_names, Ask_p_fix))

# Decision Vars

quantity_s = LpVariable.dict('S', dec_var_names, lowBound=0)                # Qauntity Short
quantity_l = LpVariable.dict('L', dec_var_names, lowBound=0)                # Qauntity Long

y = LpVariable.dict('y_', dec_var_names, lowBound=0, cat= 'Binary')
z = LpVariable.dict('z_', dec_var_names, lowBound=0, cat = 'Binary')

# Intializing the Problem
dedication_3 = LpProblem('Part 8', LpMinimize)

# Objective function
dedication_3 += lpSum([bids[i]*quantity_s[i] for i in dec_var_names]) - lpSum([asks[i]*quantity_l[i] for i in dec_var_names]) 

# Constraints

for i in range(0,len(cf_matrix[0])):
    # Liabilties constraint
    dedication_3 += lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_l[j] for j in dec_var_names]) - lpSum([period_dict['Period {}'.format(i+1)][j]*quantity_s[j] for j in dec_var_names]) == liabilities[i]/1000
    # Constraint stating that the "excess spent" at time t must be <= the current reserves.
    dedication_3 +=  lpSum([y[j] for j in dec_var_names if possibilities.loc[j,'period'] == i+1]) == lpSum([z[j] for j in dec_var_names if possibilities.loc[j,'period'] == i+1])







c:\Users\Chaz\AppData\Local\Programs\Python\Python39\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


KeyError: 'T 1.75 05/15/22'

In [ ]:
dedication_3.solve()
LpStatus[dedication_3.status]
dedication_3.objective.value()


In [ ]:
composition = pd.DataFrame(
    [v.varValue for v in dedication_2.variables() if v.varValue > 0],
    index=[str(v.name[:-8].replace('_', ' ')+v.name[-8:].replace('_', '/')) for v in dedication_2.variables() if v.varValue > 0],
    columns=['Quantity']
    )


In [ ]:
'''Manipulating the dataframe so that it will be sorted by maturity'''
maturities_ded_2 = [datetime.datetime.strptime(i[-8:], '%m/%d/%y') for i in composition.index.to_list()]

composition.reset_index(inplace=True)
composition['Dates'] = maturities_ded_2
composition.set_index('Dates',inplace=True)
composition.sort_index(axis=0, inplace=True)
composition.set_index('index', inplace = True)


In [ ]:
'''Printing Solutions'''
'''Note that the portfolio cost is a negative because our trade balance was positive'''
md(''' 
##### <center> Dedication Portfolio Cost & Composition  </center>

<center>

Portfolio Cost $ = \${:.2f} $ <br>

</center>

<center>

{}

'''.format(dedication_2.objective.value()*-1000000,composition.to_markdown(colalign = ("right",))))

# Getting CUSIPs

In [451]:
containments = {}
for i in range(1,8):
    containments[i] = pd.read_html('https://www.marketwatch.com/tools/markets/bonds/a-z/U/{}'.format(i))[0]['Name'].tolist() 


titles = []
for i in range(1,8):
    for j in containments[i]:
        if 'U.K.' not in j and 'TIPS' not in j  and 'FR ' not in j:
            titles.append(j)
        
ontherun = [i for i in titles if '(TM' in i]
offtherun1 = [i.replace('United States Treasury','T') for i in titles if '(TM' not in i]
offtherun2 = [i if 'T 0' not in i else i.replace('T 0', 'T 10') for i in offtherun1]
months = ['Jan ', 'Feb ', 'Mar ', 'Apr ', 'May ', 'Jun ', 'Jul ', 'Aug ', 'Sep ', 'Oct ', 'Nov ', 'Dec ']

offtherun3 = []
for i in offtherun2:
    for j in range(len(months)):
        if months[j] in i and j<9:
            offtherun3.append(i.replace(months[j], '0{}/'
            .format(j+1)))
        elif months[j] in i and j>=9:
            offtherun3.append(i.replace(months[j], '{}/'.format(j+1)))

offtherun4 = []
for i in offtherun3:
    a = i.replace(', 20', '/')
    b = a.replace('%', '')
    c = b.replace('-Year Note', '')
    d = c.replace('-Year Bond', '')
    e = d.replace('Bill', '0.000')
    for j in [' 0.000 ',' 2 ', ' 3 ', ' 5 ', ' 7 ', ' 10 ', ' 20 ', ' 30 ']:
        if j in e and e.replace(j, ' ') not in offtherun4 and j != ' 0.000 ':
            offtherun4.append(e.replace(j, ' '))
        elif j == ' 0.000 ' and j in e and e not in offtherun4:
            offtherun4.append(e)



uno_mas = [i.replace(' (', ' (US') for i in offtherun4]
ticks_and_cusips = [i if ' 1.25 ' not in i else i.replace(' 1.25 ', ' 1.250 ')   for i in uno_mas]
ticks = [i[:-15] for i in ticks_and_cusips]
cusips = [i[-13:-1] for i in ticks_and_cusips]

In [452]:
cusips

['US912810FG86',
 'US91282CAE12',
 'US912828ZQ64',
 'US91282CAV37',
 'US91282CBL46',
 'US91282CCS89',
 'US91282CDJ71',
 'US9128282A70',
 'US912828Z948',
 'US912828TJ95',
 'US912828YB05',
 'US912828P469',
 'US912828R366',
 'US91282CCB54',
 'US912828TY62',
 'US912828VB32',
 'US912828YS30',
 'US91282CDY49',
 'US912828K742',
 'US912828UN88',
 'US912828J272',
 'US912828U246',
 'US912828XB14',
 'US9128282R06',
 'US912828V988',
 'US912828G385',
 'US912828M565',
 'US9128283F58',
 'US912828D564',
 'US912828X885',
 'US9128286T26',
 'US912828VS66',
 'US912828WJ58',
 'US9128286B18',
 'US912828B667',
 'US9128283W81',
 'US912828WE61',
 'US9128284V99',
 'US9128284N73',
 'US91282CEP23',
 'US9128285M81',
 'US912810SQ22',
 'US912810SR05',
 'US912810ST60',
 'US912810TA60',
 'US912810SW99',
 'US912810TC27',
 'US912810SY55',
 'US912810TF57',
 'US91282CBX83',
 'US91282CAG69',
 'US91282CCU36',
 'US91282CBN02',
 'US91282CBG50',
 'US91282CAC55',
 'US91282CCN92',
 'US912828ZX16',
 'US91282CCK53',
 'US91282CBU45

In [315]:
cusips

['US912810FG86',
 'US91282CAE12',
 'US912828ZQ64',
 'US91282CAV37',
 'US91282CBL46',
 'US91282CCS89',
 'US91282CDJ71',
 'US9128282A70',
 'US912828Z948',
 'US912828TJ95',
 'US912828YB05',
 'US912828P469',
 'US912828R366',
 'US91282CCB54',
 'US912828TY62',
 'US912828SV33',
 'US912828VB32',
 'US912828YS30',
 'US91282CDY49',
 'US912828K742',
 'US912828UN88',
 'US912828J272',
 'US912828U246',
 'US912828XB14',
 'US9128282R06',
 'US912828V988',
 'US912828G385',
 'US912828M565',
 'US9128283F58',
 'US912828D564',
 'US912828X885',
 'US9128286T26',
 'US912828VS66',
 'US912828WJ58',
 'US9128286B18',
 'US912828B667',
 'US9128283W81',
 'US912828WE61',
 'US9128284V99',
 'US9128284N73',
 'US91282CEP23',
 'US9128285M81',
 'US912810SQ22',
 'US912810SR05',
 'US912810ST60',
 'US912810TA60',
 'US912810SW99',
 'US912810TC27',
 'US912810SY55',
 'US912810TF57',
 'US91282CBX83',
 'US91282CAG69',
 'US91282CCU36',
 'US91282CBN02',
 'US91282CBG50',
 'US91282CAC55',
 'US91282CCN92',
 'US912828ZX16',
 'US91282CCK53

In [316]:
matching = term_structure_df.set_index('ref_data')
matching['match'] = ['No Match' for i in range(len(matching))]
for i in ticks:
    if i in matching.index.tolist():
        matching.loc[i, 'match'] = matching.loc[i, 'px_last']

In [318]:
matching

MATURITY  COUPON      BID    ASKED  ASKED YIELD  px_last  \
ref_data                                                                      
T 1.750 05/15/22 2022-05-15   1.750  100.004  100.010      -1.9931  100.010   
T 2.125 05/15/22 2022-05-15   2.125  100.006  100.012      -2.5447  100.012   
T 0.000 05/17/22 2022-05-17   0.000    0.658    0.648       0.6570   99.352   
T 0.000 05/19/22 2022-05-19   0.000    0.670    0.660       0.6690   99.340   
T 0.000 05/24/22 2022-05-24   0.000    0.690    0.680       0.6900   99.320   
...                     ...     ...      ...      ...          ...      ...   
T 1.875 02/15/51 2051-02-15   1.875   75.290   75.310       3.1520   75.310   
T 2.375 05/15/51 2051-05-15   2.375   85.136   85.156       3.1410   85.156   
T 2.000 08/15/51 2051-08-15   2.000   78.086   78.106       3.1370   78.106   
T 1.875 11/15/51 2051-11-15   1.875   75.314   76.014       3.1240   76.014   
T 2.250 02/15/52 2052-02-15   2.250   83.114   83.134       3.1080   83.134   

                        ttm    match  
ref_data                              
T 1.750 05/15/22  -0.003251   100.01  
T 2.125 05/15/22  -0.003251  100.012  
T 0.000 05/17/22   0.002229   99.352  
T 0.000 05/19/22   0.007708    99.34  
T 0.000 05/24/22   0.021407    99.32  
...                     ...      ...  
T 1.875 02/15/51  28.772092    75.31  
T 2.375 05/15/51  29.015927   85.156  
T 2.000 08/15/51  29.267982   78.106  
T 1.875 11/15/51  29.520037   76.014  
T 2.250 02/15/52  29.772092   83.134  

[364 rows x 8 columns]

In [297]:
matching.loc['T 2.875 11/30/25']

MATURITY       2025-11-30 00:00:00
COUPON                       2.875
BID                        100.022
ASKED                      100.026
ASKED YIELD                  2.849
px_last                    100.026
ttm                       3.544694
match                      100.026
Name: T 2.875 11/30/25, dtype: object

In [317]:
matching[matching['match'] == 'No Match']

Empty DataFrame
Columns: [MATURITY, COUPON, BID, ASKED, ASKED YIELD, px_last, ttm, match]
Index: []

In [453]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
d = webdriver.Chrome('C:/Users/Chaz/chromedriver')
d.implicitly_wait(5)
inners = []


d.get('https://google.com')
e = d.find_element_by_name('q')
e.send_keys('US91282CED92 cbonds')
e.send_keys(Keys.ENTER)

elems = d.find_element_by_partial_link_text("https://cbonds.com")

elems.click()
try:
    outstanding = d.find_element_by_id("cb_bond_page_ginfo_resume_placed_volume").find_element_by_class_name("value").get_attribute('innerHTML')
    inners.append(outstanding)
except NoSuchElementException:
    new_link = d.find_element_by_partial_link_text("USA, ").click()
    outstanding = d.find_element_by_id("cb_bond_page_ginfo_resume_placed_volume").find_element_by_class_name("value").get_attribute('innerHTML')
    inners.append(outstanding)
        






C:\Users\Chaz\AppData\Local\Temp\ipykernel_35240\2243867927.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C:/Users/Chaz/chromedriver')
C:\Users\Chaz\AppData\Local\Temp\ipykernel_35240\2243867927.py:12: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  e = d.find_element_by_name('q')
C:\Users\Chaz\AppData\Local\Temp\ipykernel_35240\2243867927.py:16: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  elems = d.find_element_by_partial_link_text("https://cbonds.com")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"https://cbonds.com"}
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F7B8F3+2406643]
	Ordinal0 [0x00F0AF31+1945393]
	Ordinal0 [0x00DFC748+837448]
	Ordinal0 [0x00E292E0+1020640]
	Ordinal0 [0x00E2957B+1021307]
	Ordinal0 [0x00E56372+1205106]
	Ordinal0 [0x00E442C4+1131204]
	Ordinal0 [0x00E54682+1197698]
	Ordinal0 [0x00E44096+1130646]
	Ordinal0 [0x00E1E636+976438]
	Ordinal0 [0x00E1F546+980294]
	GetHandleVerifier [0x011E9612+2498066]
	GetHandleVerifier [0x011DC920+2445600]
	GetHandleVerifier [0x01014F2A+579370]
	GetHandleVerifier [0x01013D36+574774]
	Ordinal0 [0x00F11C0B+1973259]
	Ordinal0 [0x00F16688+1992328]
	Ordinal0 [0x00F16775+1992565]
	Ordinal0 [0x00F1F8D1+2029777]
	BaseThreadInitThunk [0x75C0FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77D07A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77D07A4E+238]


In [429]:
inners

[]

In [459]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
d = webdriver.Chrome('C:/Users/Chaz/chromedriver')
d.implicitly_wait(10)
inners = []


d.get('https://cbonds.com')
for CUSIP in cusips:
    e = d.find_element_by_id('mainSearch')
    e.send_keys(CUSIP)

    elems = d.find_element_by_class_name("result").find_element_by_class_name("list")

    elems.click()

    outstanding = d.find_element_by_id("cb_bond_page_ginfo_resume_placed_volume").find_element_by_class_name("value").get_attribute('innerHTML')
    inners.append(outstanding)
        

        




    #scrape page here 


C:\Users\Chaz\AppData\Local\Temp\ipykernel_35240\3210868441.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('C:/Users/Chaz/chromedriver')
C:\Users\Chaz\AppData\Local\Temp\ipykernel_35240\3210868441.py:14: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  e = d.find_element_by_name('q')
C:\Users\Chaz\AppData\Local\Temp\ipykernel_35240\3210868441.py:17: DeprecationWarning: find_element_by_partial_link_text is deprecated. Please use find_element(by=By.PARTIAL_LINK_TEXT, value=link_text) instead
  elems = d.find_element_by_partial_link_text("https://cbonds.com")


AttributeError: 'NoneType' object has no attribute 'click'

In [447]:
inners

['<span style="user-select: all;">11,349,444,000 USD</span>',
 '<span style="user-select: all;">133,018,661,000 USD</span>',
 '<span style="user-select: all;">109,879,724,700 USD</span>',
 '<span style="user-select: all;">133,680,823,700 USD</span>',
 '<span style="user-select: all;">140,062,692,000 USD</span>',
 '<span style="user-select: all;">142,197,430,800 USD</span>',
 '<span style="user-select: all;">144,644,350,400 USD</span>',
 '<span style="user-select: all;">653,562,405,000 USD</span>',
 '<span style="user-select: all;">88,113,080,200 USD</span>',
 '<span style="user-select: all;">65,994,896,300 USD</span>',
 '<span style="user-select: all;">92,619,002,100 USD</span>',
 '<span style="user-select: all;">64,946,659,900 USD</span>',
 '<span style="user-select: all;">69,496,759,100 USD</span>',
 '<span style="user-select: all;">148,499,896,200 USD</span>',
 '<span style="user-select: all;">121,993,356,400 USD</span>',
 '<span style="user-select: all;">119,929,077,900 USD</span>'

In [404]:
print(outstanding)

<span style="user-select: all;">11,349,444,000 USD</span>


In [ ]:
d.find_element_by_partial_link_text("https://cbonds").get_attribute('href').click()
elems = d.find_element_by_class_name("result").find_element_by_xpath('//a[contains(@href, "/bonds/")]').get_attribute('href')
finder = d.find_element_by_partial_link_text("Bonds")

In [387]:
print(elems)

https://cbonds.com/bonds/36939/


In [ ]:
print(finder)

In [ ]:
d.find_element_by_css_selector("ul.list [href]").get_attribute('href')
d.find_element_by_partial_link_text("https://cbonds.com/bonds/")

In [ ]:
for CUSIP in cusips:
    d.get('https://cbonds.com/')
    e = d.find_element_by_id('mainSearch')
    e.send_keys(CUSIP)
    e.send_keys(Keys.ENTER)
    #scrape page here 